In [14]:
from docx import Document
import zipfile
from lxml import etree
import xml.etree.ElementTree as ET

In [27]:
def extract_text_from_tree(tree, namespaces):
    # Extract text from the XML tree, handling the updated tags
    text_elements = tree.xpath('//w:t', namespaces=namespaces)
    text_content = []
    for elem in text_elements:
        text_content.append(elem.text)
    return ''.join(text_content)

In [142]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

def enclose_elements(input_file, output_file, namespaces):
    try:
        # Parse the XML file
        tree = ET.parse(input_file)
        root = tree.getroot()

        # Function to recursively process elements
        def process_element(elem):
            new_elem = None
            if elem.tag == f"{namespaces['w']}:p":
                new_elem = ET.Element('para')
                new_elem.text = ''.join(elem.itertext())  # Collect all text within the paragraph
            elif elem.tag == f"{namespaces['w']}:instrText":
                new_elem = ET.Element('field')
                new_elem.text = elem.text
            else:
                new_elem = ET.Element(elem.tag, elem.attrib)
                new_elem.text = elem.text
            
            # Process child elements
            for child in elem:
                if new_elem is not None:
                    new_elem.append(process_element(child))
            
            return new_elem

        # Create a new root element for the output
        new_root = process_element(root)

        # Convert the ElementTree to a string
        xml_str = ET.tostring(new_root, encoding='utf-8')

        # Parse the string with minidom for pretty printing
        dom = xml.dom.minidom.parseString(xml_str)
        pretty_xml_as_str = dom.toprettyxml(indent="  ")

        # Write the indented XML to the output file
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(pretty_xml_as_str)

    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [163]:
import xml.etree.ElementTree as ET

def extract_paragraphs(input_xml, output_xml):
    # Define the namespace dictionary
    namespaces = {'w': 'http://schemas.microsoft.com/office/word/2003/wordml'} 

    # Parse the input XML file
    tree = ET.parse(input_xml)
    root = tree.getroot()

    # Create a new root element for the output XML
    new_root = ET.Element('root')

    # Find all paragraphs with namespace 'w:p'
    for para in root.findall('.//w:p', namespaces):
        # Create a new <para> element
        new_para = ET.Element('para')
        
        # Append the paragraph text or elements to the new <para> element
        # Here, we assume you want the text content
        new_para.text = ''.join(para.itertext())
        
        # Append the new <para> to the new root
        new_root.append(new_para)

    # Convert the ElementTree to a string
    rough_string = ET.tostring(new_root, encoding='utf-8')

    # Use minidom to pretty print the XML
    reparsed = xml.dom.minidom.parseString(rough_string)
    pretty_xml_as_string = reparsed.toprettyxml(indent="  ")

    # Write the prettified XML to the output file
    with open(output_xml, 'w', encoding='utf-8') as f:
        f.write(pretty_xml_as_string)

    # # Write the new XML tree to the output file
    # new_tree = ET.ElementTree(new_root)
    # new_tree.write(output_xml, encoding='utf-8', xml_declaration=True)


import xml.etree.ElementTree as ET
import xml.dom.minidom

def extract_paragraphs_fields_with_bold(input_xml, output_xml):
    # Define the namespace dictionary
    namespaces = {'w': 'http://schemas.microsoft.com/office/word/2003/wordml'} 

    # Parse the input XML file
    tree = ET.parse(input_xml)
    root = tree.getroot()

    # Create a new root element for the output XML
    new_root = ET.Element('root')

    # Find all paragraphs with namespace 'w:p'
    for para in root.findall('.//w:p', namespaces):
        # Create a new <para> element
        new_para = ET.Element('para')

        # Iterate through the children of the paragraph
        for run in para.findall('.//w:r', namespaces):
            # Check if the run has a bold property
            is_bold = run.find('.//w:rPr/w:b-cs', namespaces) is not None

            for elem in run:
                if elem.tag == '{http://schemas.microsoft.com/office/word/2003/wordml}instrText':
                    # If the element is an instrText, wrap it in <field>
                    field_elem = ET.Element('field')
                    field_elem.text = ''.join(elem.itertext())
                    new_para.append(field_elem)
                else:
                    # Join the text content
                    text_content = ''.join(elem.itertext())
                    # Check if the text content is not empty
                    if text_content.strip():
                        # Create a new element for text content
                        if is_bold:
                            # Wrap text in <bold> if bold is detected
                            bold_elem = ET.Element('bold')
                            bold_elem.text = text_content
                            new_para.append(bold_elem)
                        else:
                            text_elem = ET.Element('text')
                            text_elem.text = text_content
                            new_para.append(text_elem)

        # Append the new <para> to the new root
        new_root.append(new_para)

    # Convert the ElementTree to a string
    rough_string = ET.tostring(new_root, encoding='utf-8')

    # Use minidom to pretty print the XML
    reparsed = xml.dom.minidom.parseString(rough_string)
    pretty_xml_as_string = reparsed.toprettyxml(indent="  ")

    # Write the prettified XML to the output file
    with open(output_xml, 'w', encoding='utf-8') as f:
        f.write(pretty_xml_as_string)

# Example usage:
# extract_paragraphs_fields_with_bold('input.xml', 'output.xml')

input_xml = r"NYLB\NYLB 32 (revision copy).xml" # Replace with your input XML file path
output_xml = 'temp_files/temp_output.xml'  # Replace with your desired output XML file path
extract_paragraphs_fields_with_bold(input_xml, output_xml)

In [33]:
part = ['II. Agricultural Cooperative Corporations\nA. Form Drafting Principles\n32:8. Introduction\nResearch References\n ►<field>tk.ref.id="I04AAC5"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>A nonprofit cooperative agricultural corporation may be organized to engage in any cooperative activities for its members, including the marketing, selling of, bargaining, and contracting for agricultural products and the by-products of such products.1 Any five or more producers of agricultural products, to include cooperative corporations of such producers organized under the laws of New York or any other state, may form a nonprofit cooperative agricultural corporation, either with or without capital stock.2\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>If the principal activities of the corporation are connected with the marketing, processing, manufacture, or sale of agricultural products, it may be termed a cooperative agricultural corporation and incorporated as such.3 If its principal activities relate to the purchase of supplies for producers of agricultural products, it may be termed a cooperative purchasing association and may be incorporated as such.4 In either case, the corporation may engage in both types of activity.5 A nonprofit agricultural cooperative association of this nature may be incorporated with or without capital stock.6\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>The amount of dividends that may be declared on stock cannot exceed 12&percnt;% per year.7\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>The exempt income tax status of farmers\' cooperatives is extended even though they have capital stock if the dividend rate of the stock is fixed at a rate not to exceed the legal rate of interest in the state of incorporation or 8&percnt;% per year, whichever is greater, on the value of the consideration for which the stock was issued and if substantially all such stock (other than nonvoting preferred stock, the owners of which are not entitled or permitted to participate, directly or indirectly, in the profits of the association, on dissolution or otherwise, beyond the fixed dividends) is owned by producers who market their products or purchase their supplies and equipment through the association.8\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>Transactions with nonmembers will not bar a farmers\' cooperative from claiming an exempt status under the federal income tax laws, provided the value of the transactions, in relation to those with members, does not exceed statutory limits.9 An exemption will not be denied any association that markets the products of nonmembers in an amount the value of which does not exceed the value of the products marketed for members or which purchases supplies and equipment for nonmembers in an amount the value of which does not exceed the value of the supplies and equipment purchased for members, provided the value of the purchases made for persons who are neither members nor producers does not exceed 15&percnt;% of the value of all purchases.10\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>Business done for the United States or any of its agencies is disregarded in determining the right to exemption.11\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>The provisions of the Business Corporation Law of New York and all powers and rights conferred under such law apply to agricultural cooperative corporations except where such provisions are in conflict with or are inconsistent with the express provisions of the Agricultural Cooperative Corporations Law.12\n32:9. Drafting preorganization agreements and certificates of incorporation\nResearch References\n ►<field>tk.ref.id="I047HFH"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|9|32:9"</field>In drafting the organization agreement, also commonly known as the preorganization agreement, the general policies of promoting the cultivation and marketing of agricultural products through cooperation and of distributing the agricultural products in as direct a manner as possible between the producer and the consumer should be stated.1\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|9|32:9"</field>It is the declared policy of the state of New York to promote the effective production and merchandising of agricultural commodities by providing the means by which farmers may act together in manufacturing, processing, preparing for market, handling and/or marketing their farm products and by enabling farmers to act together in purchasing, testing, grading, processing, distributing and/or furnishing farm supplies and/or farm business services through cooperatives operated for the mutual benefit of the members of the corporations as producers and purchasers.2\nPractice Tip:\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|9|32:9"</field>As eventually drawn, the agreement should bind the producer to belong to and patronize the proposed cooperative and to furnish the specified amount of initial capital, contingent on the signing up of a sufficient number of other prospective members within a given time period. The chief provisions proposed for the bylaws should be indicated and the form of the marketing agreement to be used should be set forth.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|9|32:9"</field>New York statutory law specifies the matters required to be included in the certificate of incorporation of a cooperative corporation.3 These requirements are set out in a subsequent checklist.4 The certificate should specify that the organization will deal only in agricultural products, as defined in ►<field>st.ref.id="I05PANM"</field>Sections 110 to 113 of the New York Cooperative Corporations Law,5 and should make clear that the organization will qualify as an agricultural cooperative within the meaning of the statute.6 &ldquo;“Agricultural product&rdquo;” is defined as any product of cultivating land, and includes floricultural, horticultural, viticultural, forestry, nut, dairy, livestock, poultry, bee, and any farm products or by-products thereof.7 An &ldquo;“agricultural cooperative&rdquo;” is a cooperative, either stock or nonstock, operated for the mutual benefit of the members in which no member is allowed more than one vote because of the amount of stock or membership capital he or she may own therein; the cooperative does not pay dividends on stock or membership capital in excess of 12&percnt;% per year; and the cooperative does not deal in farm products, agricultural waste products or agricultural compost, farm supplies, farm business services and the capture of methane and other gases for the generation and use or sale of energy, with or for nonmembers in an amount greater in value than the total amount of such business transacted by it with or for members.8\nPractice Tips:\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|9|32:9"</field>Because the certificate of incorporation of an agricultural cooperative association is generally more difficult to amend than its bylaws,9 it is recommended that the substantive content of the certificate be limited to the statutory requirements.10\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|9|32:9"</field>In general, the provisions of the New York Business Corporation Law applies to agricultural cooperative corporations except where such provisions are in conflict with or are inconsistent with the particular provisions applicable to cooperatives.11\n32:10. Drafting certificate of incorporation; formal requirements\nResearch References\n ►<field>tk.ref.id="I05PAMA"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|10|32:10"</field>The certificate of incorporation must be made, acknowledged, and filed by five or more persons.1 All the subscribers must be of full age, at least two-thirds of them must be citizens of the United States and at least one of them must be a resident of the State of New York and, of the persons named as directors at least one must be a citizen of the United States and a resident of the State of New York.2 It may contain any provision, not inconsistent with the law, that the incorporators elect to set forth for the regulation of the internal affairs of the corporation.3 The general requirements for a certificate of incorporation to be filed by a cooperative corporation are set out in the Cooperative Corporations Law.4\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|10|32:10"</field>To assist in the processing of documents that are to be filed in the office of the Department of State, the physical appearance and makeup of such documents, including certificates of incorporation, should be in good legal form and be clearly legible.5\nPractice Tip:\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|10|32:10"</field>It is recommended that you check with the Secretary of State\'s office with regard to the form, filing, and information necessary for the return of the filed certificate.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|10|32:10"</field>The directors are required periodically to set aside reasonable sums for reserves.6 The net margins or net retained proceeds may, in the discretion of the directors, be distributed at least once a year to members or patrons by uniform distribution and calculated on such basis as the bylaws or marketing contract may prescribe.7 Distributions may be credited on account of the issuance to members or patrons of capital stock or other securities of the corporation.8 In the case of cooperatives with capital stock, dividends may not exceed 12&percnt;% per year on any class of stock.9\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|10|32:10"</field>A certificate of incorporation or amendment of such certificate will not take effect until it has been filed with the Secretary of State.10 The Secretary of State will provide copies of the certificates or amendments to the Commissioner of Agriculture and Markets or his or her designee on request by corporation name.11\n32:11. Drafting bylaws\nResearch References\n ►<field>tk.ref.id="I047PJ6"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04AIO2"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|11|32:11"</field>The New York Statutes require that bylaws regulating the conduct and manner of the corporation be adopted by its incorporators at the organization meeting.1 Thereafter, bylaws may be adopted by vote of the holders of the shares at the time entitled to vote in the election of any directors.2 When so provided in the certificate of incorporation or a bylaw adopted by the shareholders, bylaws may also be adopted by the board of directors.3\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|11|32:11"</field>Any bylaw adopted by the board of directors must be reported to the annual meeting of the corporation and, if not affirmatively approved at such meeting, will cease to be in effect.4 A listing of the matters authorized by statute to be included in the bylaws of an agricultural cooperative association is set out in a subsequent checklist.5\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|11|32:11"</field>In many respects, the bylaws of an agricultural cooperative resemble those of a business corporation.6 However, there are several notable differences. For example, the bylaws of an agricultural marketing cooperative are often tied to a marketing agreement between the cooperative and its members. The bylaws frequently incorporate the agreement by reference and may specify the method to be used to create, accumulate, and maintain the reserves required by statute.7\nB. Checklists\n32:12. Checklist-- Information required in certificate of incorporation of agricultural cooperative corporation\nCOL\nCHK\nFCV\nFTR\nCOR\nEBL\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>1. Name of corporation, which must include word &ldquo;“Cooperative.&rdquo;”\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>2. Statement of purpose or purposes for which cooperative is formed, which shall be one or more of the objects enumerated in the N.Y. Cooperative Corporations Law.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>3. City, village or town and county in which its office is to be located.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>4. Term of existence of corporation.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>5. Names and post office addresses of incorporators.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>6. Statement of whether organized with or without capital stock.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>7. If organized with capital stock.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>a. Statement of total authorized capital stock, number and types of shares, and par value of shares.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>b. Rights, preferences, and restrictions of each type of share.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>c. If preferred stock is issued, a limitation on dividends that will not exceed 12&percnt;% per year.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>8. Number of its directors or that number of directors will be within a stated minimum and maximum as stated in bylaws. In any event, number may not be less than five.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>9. Names and post office addresses of directors until first annual meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>10. Statement that all subscribers are of full age, that at least two-thirds of them are citizens of the United States, that at least one of them is a resident of New York, and that of the persons named as directors at least one is a citizen of the United States and a resident of the State of New York.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>11. Designation of Secretary of State as agent of corporation upon whom process may be served and post office address within or without New York to which Secretary of State may mail a copy of any process against it served on such official.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>12. If corporation is to have a registered agent, his or her name and address within New York and a statement that the registered agent is to be the agent of the corporation upon whom process against it may be served.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>13. Any other provision(s) consistent with law with respect to management, regulation, government, financing, indebtedness, membership, issuance, retirement, and transfer of stock, if required, or any provisions relative to way or manner in which cooperative corporation will operate or with respect to its members, officers, or directors and any other provisions relating to its affairs.\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>See ►<field>st.ref.id="I05RVNJ"</field>Coop. Corp. Law &s;§11 for discussion of the required contents of the certificate.\nResearch References\n ►<field>tk.ref.id="I05P2IN"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05M9DR"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I05RVHW"</field>N.Y. Jur. 2d, Business Relationships &s;§73\nItem 1.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>Where copy of certificate of incorporation was not included in record of shareholders\' action contesting their disqualification from voting for board of directors and where corporation\'s name did not include word &ldquo;“cooperative,&rdquo;” it was presumed that corporation was established under Business Corporation Law rather than Cooperative Corporations Law. ►<field>cc.ref.id="I05P2D0"</field>Yu v. Linton, 68 A.D.2d 856, 414 N.Y.S.2d 558 (1st Dep\'t 1979).\nResearch References\n ►<field>rc.ref.id="I05M984"</field>N.Y. Jur. 2d, Business Relationships &ss;§§164 to 179\nItem 2.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>If the principal activities of a cooperative agricultural corporation are connected with the marketing, processing, manufacture, sale, or other dispositions of agricultural products, agricultural waste product, or agricultural compost, including the capture of methane and other gases for the generation and use or sale of energy, as defined in ►<field>st.ref.id="I05JG38"</field>&s;§1-103 of the N.Y. Energy Law, it may be termed a cooperative marketing corporation and incorporated as such. If its principal activities relate to the purchase of supplies for producers of agricultural products, it may be termed a cooperative purchasing association and incorporated as such, but, in either case such a corporation may engage in both such lines of activity. Nothing in ►<field>st.ref.id="I05RVHT"</field>Article 6 of the Cooperative Corporations Law or ►<field>st.ref.id="I05P2CX"</field>&s;§112 of N.Y. Cooperative Corporations Law will be deemed to prohibit the incorporation of a cooperative corporation under Article 2 of the Cooperative Corporations Law or affect the existing powers of any existing cooperative corporation not within Section 111 of the Cooperative Corporations Law. ►<field>st.ref.id="I05M981"</field>Coop. Corp. Law &s;§112.\nResearch References\n ►<field>rc.ref.id="I05JG35"</field>N.Y. Jur. 2d, Business Relationships &s;§61\nItem 4.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>Subject to any limitations provided by statute or in a certificate of incorporation, a corporation has the power, in furtherance of its corporate purposes, to have perpetual duration. ►<field>st.ref.id="I05GMY9"</field>BCL &s;§202(a)(1).\nResearch References\n ►<field>rc.ref.id="I05RVHR"</field>N.Y. Jur. 2d, Business Relationships &s;§110\nItem 11.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|12|32:12"</field>The certificate of incorporation must contain a designation of the Secretary of State as agent of the corporation for service of process and provide therein a post office address to which the Secretary of State can mail a copy of any process against the corporation served on the Secretary of State. ►<field>st.ref.id="I05P2CV"</field>Coop. Corp. Law &s;§11(10).\nResearch References\n ►<field>rc.ref.id="I05M97Z"</field>N.Y. Jur. 2d, Business Relationships &s;§73\n ►<field>rc.ref.id="I047PW3"</field>N.Y. Jur. 2d, Business Relationships &ss;§§1062, 1063\n32:13. Checklist-- Matters that may be covered in bylaws of agricultural cooperative\nCOL\nNFP\nCHK\nFCV\nFTR\nEBL\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>1. Name of cooperative.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Principal office.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Other offices.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>2. Purposes and objectives of cooperative.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Method of accomplishing.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) In accordance with all laws and regulations.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(c) In compliance with requirements of the Internal Revenue Code to qualify for cooperative status.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Business with nonmembers shall not exceed business done with members.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>3. Membership.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Eligibility for membership or to purchase stock.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Method of joining.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) Application.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(2) Registration fee.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(3) Subscription for membership or stock.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(A) Installment payments.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Interest payable.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(B) Full payment.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(C) Patronage payments, dividends, and other refunds credited to member as payment on share or stock until fully paid.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(4) Approval of board of directors.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(c) Voting rights per membership share or shares of stock.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(d) Payment of patronage dividends and other distributions made on basis of membership share.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) All payments in compliance with Internal Revenue Code.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(2) Payments made in cash unless member directs payment to purchase of additional shares.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(A) Limitation on number of membership shares or shares of stock.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(B) Suspension and removal of members.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(i) Grounds.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(ii) Procedure.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(e) Membership interests and stock not transferable.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) Exception for transfers to family members.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Approval of board of directors.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(f) Membership shares and stock are nonassessable.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>4. Withdrawal of amounts credited on purchase of membership or stock shares.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Withdrawal upon written request.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Cooperative has until one year from date of request to pay.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(c) Payment in cash, goods, or services of cooperative.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(d) Partial withdrawal to result in proportionate reduction of credits and distributions.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>5. Distributions.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Patronage refunds based on purchases.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Dividends on capital limited to greater of 8&percnt;% or legal rate of interest.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(c) Any capital impairment restored before distributions are made.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(d) Payment of earnings of cooperative based on purchases and business done with cooperative.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>6. Each member to include patronage dividends and other distributions in income in accordance with Internal Revenue Code.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>7. Meetings of members.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Annual meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) Purposes.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(2) Date, time, location.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(3) Notices.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Special meetings.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) Method for calling.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(2) Purpose.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(3) Notices.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(c) Quorum for meetings.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(d) Conduct of meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(e) Proxies.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(f) Method of voting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>8. Directors.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Number (not less than five), tenure, qualifications.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) Member of cooperative in good standing.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Time and manner of nominating and choosing.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(c) Qualifications.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(d) Term of office.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(e) Election at annual meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(f) Number, not less than a majority, needed to exercise their powers.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(g) Powers and duties.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(h) Prohibition of self-dealing.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(i) Compensation.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(j) Method for removing director.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) Grounds for removal.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(2) Procedure.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(k) Meetings of directors.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Regular meetings.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>9. Officers.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Titles.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(1) President (appointed from directors).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(2) One or more vice presidents (appointed from directors).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(3) Secretary and treasurer, or secretary-treasurer (need not be directors or members, but must be elected by members).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Tenure.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(c) Powers and duties.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(d) Compensation.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(e) Manner of appointment and removal.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>10. Amount each member or shareholder will pay annually or from time to time, if at all, to carry on business of association and charge, if any, to be paid by patrons for services rendered.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>11. Penalties for violation of bylaws.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>12. Form of marketing contract between corporation and its members or shareholders (which contract will be binding on every member or shareholder unless otherwise agreed to in writing).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>13. Manner of adoption, amendment, or repeal of bylaws.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) By directors.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) By members or shareholders.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>14. Date of commencement of corporation\'s business or fiscal year.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>15. Manner in which dissolution of corporation may be effectuated and division and distribution of remaining funds.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>16. Such other provisions as are proper and necessary in order to carry out purposes for which corporation is formed.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(a) Bonding of officers and employees.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>(b) Annual audit and report.\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>As to matters that are suggested by statute for inclusion in the bylaws of a nonprofit cooperative corporation, see ►<field>st.ref.id="I047Q39"</field>Coop. Corp. Law &s;§14(h), (i).\nResearch References\n ►<field>tk.ref.id="I04DCD1"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I047Q4E"</field>West\'s Key Number Digest, Associations &key;128, 129\nItem 1.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Where copy of certificate of incorporation was not included in record of shareholders\' action contesting their disqualification from voting for board of directors and where corporation\'s name did not include word &ldquo;“cooperative,&rdquo;” it was presumed that corporation was established under Business Corporation Law rather than Cooperative Corporations Law. ►<field>cc.ref.id="I04AJ9A"</field>Yu v. Linton, 68 A.D.2d 856, 414 N.Y.S.2d 558 (1st Dep\'t 1979).\nResearch References\n ►<field>rc.ref.id="I04DCE6"</field>N.Y. Jur. 2d, Business Relationships &ss;§§164 to 179\nItem 2.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>If the principal activities of a cooperative agricultural corporation are connected with the marketing, processing, manufacture, sale, or other dispositions of agricultural products, agricultural waste product, or agricultural compost including the capture of methane and other gases, for the generation and use or sale of energy, as defined in ►<field>st.ref.id="I047Q4G"</field>&s;§1-103 of the N.Y. Energy Law, it may be termed a cooperative marketing corporation and incorporated as such. If its principal activities relate to the purchase of supplies for producers of agricultural products, it may be termed a cooperative purchasing association and incorporated as such, but in either case such a corporation may engage in both such lines of activity. Nothing in Article 6 or ►<field>st.ref.id="I04AJ9C"</field>&s;§112 of the N.Y. Cooperative Corporations Law will be deemed to prohibit the incorporation of a cooperative corporation under article two of this chapter or affect the existing powers of any existing cooperative corporation not within Section 111 of Article 6 of the Cooperative Corporations Law. ►<field>st.ref.id="I04DCE8"</field>Coop. Corp. Law &s;§112.\nResearch References\n ►<field>rc.ref.id="I05RV3O"</field>N.Y. Jur. 2d, Business Relationships &s;§61\nItem 3.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Only persons engaged in the production of agricultural products or cooperative corporations of such producers organized under the laws of New York or any other state shall be eligible for membership in any agricultural marketing or purchasing corporation formed or operated under the provisions of this article. The terms and conditions of membership shall be prescribed in the bylaws. ►<field>st.ref.id="I05P1YS"</field>Coop. Corp. Law &s;§111(a).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Every cooperative corporation shall issue to each member, upon full payment therefor, a certificate of membership (whether evidenced by stock or otherwise) which shall not be transferable otherwise than as may be prescribed in this chapter, the certificate of incorporation, and bylaws. The corporation may accept as full or partial payment a member\'s promissory note but shall hold the certificate as security for payment, without, however, affecting the member\'s right to vote unless such note is past due. See ►<field>st.ref.id="I05M8TW"</field>Coop. Corp. Law &s;§40.\nResearch References\n ►<field>rc.ref.id="I05JFP0"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\n ►<field>rc.ref.id="I05RV3M"</field>N.Y. Jur. 2d, Business Relationships &s;§825\nItem 6.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>For discussion of patronage dividends and other distributions, see ►<field>x.ref.id="I05P1YQ"</field>&s;§32:6.\nResearch References\n ►<field>rc.ref.id="I05M8TU"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\nItem 8.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>The board of directors must be chosen at the time and place and for terms fixed by the bylaws, but at least one-fourth of the directors must be elected annually. ►<field>st.ref.id="I05JFOY"</field>Coop. Corp. Law &s;§111(a).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>A cooperative corporation may provide a fair remuneration for its directors and for members of its executive committee. No director or member of the executive committee shall be a party to a contract for profit with the corporation that in substance shall differ in any way from similar contracts by it with members or with others or that shall vary from terms generally current in the district. ►<field>st.ref.id="I05GMK2"</field>Coop. Corp. Law &s;§62.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Any member may bring charges against a director by filing them in writing with the secretary, together with a petition signed by 5&percnt;% of the members, requesting removal. The corporation may thereupon remove the director by the affirmative vote of three-fourths of the members voting thereon at a meeting promptly held after due notice in writing, setting forth accurately the purpose for which such meeting is called, provided that at such meeting not less than 10&percnt;% of the entire membership vote, personally or by mail. The director involved shall be given a copy of the charges reasonably in advance of the meeting, and he or she and the complainant shall have an opportunity at the meeting to be heard in person or by counsel and to present and cross-examine witnesses. ►<field>st.ref.id="I05RV3J"</field>Coop. Corp. Law &s;§63.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>In case the bylaws provide for election of directors by districts with primary elections in each district, then the petition for removal of a director must be signed by 20&percnt;% of the members residing in the district from which he or she was elected. The board of directors must call a special meeting of the members residing in that district to consider the removal of the director in the manner above provided, and, by a concurrent vote of a majority of the members of that district voting thereon, such director shall be removed from office. See ►<field>st.ref.id="I05P1YN"</field>Coop. Corp. Law &s;§63.\nResearch References\n ►<field>rc.ref.id="I05RV3H"</field>N.Y. Jur. 2d, Business Relationships &ss;§§545, 569, 791\nItem 9.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>The corporation may provide a fair remuneration for its officers. No officer shall be a party to a contract for profit with the corporation that, in substance, shall differ in any way from similar contracts by it with members or with others or that shall vary from terms generally current in the district. ►<field>st.ref.id="I05P1YL"</field>Coop. Corp. Law &s;§62.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>The officers of every cooperative corporation must include a president, one or more vice presidents, secretary, and treasurer who are required to be elected annually by the board of directors. The president and a first vice president must be members of the corporation or of a member corporation and must be elected from among the directors. Other officers need not be directors or members of the corporation. The offices of secretary and treasurer may be combined. ►<field>st.ref.id="I05M8TP"</field>Coop. Corp. Law &s;§64.\nResearch References\n ►<field>rc.ref.id="I05JFOT"</field>N.Y. Jur. 2d, Business Relationships &ss;§§532, 536, 789\nItem 11.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>In accordance with provisions in the bylaws, any person will forfeit his or her membership upon proof that the person has ceased to be engaged in the occupation or occupations for the servicing of which the corporation was formed or has ceased to have the qualifications requisite for membership; in addition, the member must upon such notice and terms, as may be prescribed in the bylaws, surrender his or her membership certificate or shares of stock upon payment of the par or otherwise designated value thereof and of any accrued dividends thereon, as may appear in the accounting at the end of the current fiscal year. ►<field>st.ref.id="I05RV2B"</field>Coop. Corp. Law &s;§42.\nResearch References\n ►<field>rc.ref.id="I05P1XF"</field>N.Y. Jur. 2d, Business Relationships &s;§836\nItem 13.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>For an agricultural cooperative corporation, the bylaws may provide for their amendment by the board of directors, but any amendment adopted by the board that affects the preferential rights or obligations of the members or stockholders shall be reported to the next annual meeting of the corporation and, if not affirmatively approved thereat, shall cease to be in effect. Such bylaws reported to the annual meeting may be adopted, repealed, or amended on the affirmative vote of two-thirds of the members, stockholders, or delegates voting thereon at a meeting held after due written notice setting forth the proposed action and the purpose of the meeting. ►<field>st.ref.id="I05M8SJ"</field>Coop. Corp. Law &s;§111(a).\nResearch References\n ►<field>rc.ref.id="I05JFNN"</field>N.Y. Jur. 2d, Business Relationships &s;§128\nItem 15.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>A cooperative corporation may, at any meeting and upon due and express notice previously given, by vote of two-thirds of all of the members or stockholders voting thereon, discontinue its operations and settle its affairs. ►<field>st.ref.id="I05GMIR"</field>Coop. Corp. Law &s;§17.\nResearch References\n ►<field>rc.ref.id="I05RV29"</field>N.Y. Jur. 2d, Business Relationships &ss;§§1337, 1432\nItem 16(a).\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Before handling funds or securities of the corporation, amounting to $1,000 or more in any one year, the officer or employee shall be covered by an adequate bond to be approved by the board of directors. ►<field>st.ref.id="I05P1XD"</field>Coop. Corp. Law &s;§65.\nResearch References\n ►<field>rc.ref.id="I047QBM"</field>N.Y. Jur. 2d, Business Relationships &s;§687\nItem 16(b).\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|13|32:13"</field>Immediately after the close of each fiscal year, every cooperative corporation shall cause an audit to be made of its operations for such fiscal year. A written report of the audit, including a statement of services rendered by the corporation, with total amount of business transacted, balance sheet, income, and expenses shall be submitted to the annual meeting of the corporation and shall at all times be available for inspection by any member. Such audit shall be made by an experienced bookkeeper or accountant or firm of accountants not regularly employed by the corporation, provided that in the case of a cooperative corporation, the annual business of which amounts to less than $100,000, the audit may be made by an auditing committee of three members or stockholders of the corporation who shall not be directors, officers, or employees thereof. Any person violating or failing to comply with the provisions of this section shall be deemed guilty of a misdemeanor. ►<field>st.ref.id="I04AJGI"</field>Coop. Corp. Law &s;§75.\nResearch References\n ►<field>rc.ref.id="I04DCLE"</field>N.Y. Jur. 2d, Business Relationships &s;§379\n32:14. Checklist-- Survey of matters considered by planning committee of poultry cooperative\nCOL\nNFP\nCHK\nFCV\nFTR\nEBL\n ►<field>form.du="0"</field><field>form.fid="Ia3929dd0181a11e68738d3db3a7c8dd3"</field>►<field>tbl.date.updated="0"</field><field>tbl.ident="I03BXD6"</field><field>tbl.orient="port"</field><field>tbl.position="l"</field><field>tbl.table.width="100"</field><field>tbl.uuid="I039T3D"</field>\n►<field>table.resize="minus1"</field><field>table.tabstyle="text"</field>\n►<field>tgroup.align="left"</field><field>tgroup.cols="5"</field>\n►<field>colspec.colname="1"</field><field>colspec.colnum="1"</field><field>colspec.colwidth="1*"</field>\n►<field>colspec.colname="2"</field><field>colspec.colnum="2"</field><field>colspec.colwidth="1*"</field>\n►<field>colspec.colname="3"</field><field>colspec.colnum="3"</field><field>colspec.colwidth="4*"</field>\n►<field>colspec.align="right"</field><field>colspec.colname="4"</field><field>colspec.colnum="4"</field><field>colspec.colwidth="1*"</field>\n►<field>colspec.align="right"</field><field>colspec.colname="5"</field><field>colspec.colnum="5"</field><field>colspec.colwidth="1*"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.nameend="3"</field><field>entry.namest="1"</field><field>entry.resize="same"</field><field>entry.style="block"</field>I. Operating Costs and Volume of Business:\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>1. How much will the operating costs be and how will they be met? In answering, consider the following costs:\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.r</field><field>esize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize</field><field>="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.colname="3"</field><field>entry.resize</field><field>="same"</field><field>entry.style="block"</field>Rental\n\n►<field>entry.resize="same"</field><field>entry.style</field><field>="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Monthly rent\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Salaries per month:\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Manager\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Delivery\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Candlers\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Clerk\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry</field><field>.style="block"</field>&ballot;[] Pickup \n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Other costs per month:\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.nameend="4"</field><field>entry.namest="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Trucks (operation and depreciation)\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.nameend="4"</field><field>entry.namest="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Electricity, water, telephone\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.nameend="4"</field><field>entry.namest="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Supplies (fillers and flats--cases)\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Seals\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.r</field><field>esize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style</field><field>="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colnam</field><field>e="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Taxes\n\n►<field>entry.resize="same"</field><field>entry.style</field><field>="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.nameend="4"</field><field>entry.namest="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Workers\' compensation insurance\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.nameend="4"</field><field>entry.namest="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Operating expenses--manager\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Audit\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] All others cost\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.colname="5"</field><field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>2. What is the necessary business volume needed to meet these costs? In answering, please consider:\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>What is the least volume necessary to meet overhead costs?\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>Are there enough producers willing to join a cooperative to guarantee the minimum volume?\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>3. How should supplies bought by members be treated. In answering, please consider:\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>What will be the policy on purchasing supplies for members?\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updat</field><field>ed="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="ri</field><field>ght"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="s</field><field>ame"</field><field>entry.style="block"</field>Price to members?\n\n►<field>entry.resize="s</field><field>ame"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="s</field><field>ame"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.colname="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>Delivery to members?\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>4. How should delivery of member produce be handled? In answering, please consider:\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>Will members\' produce be picked up at the farm?\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>Will members\' produce be delivered to cooperative?\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>Will members\' produce be delivered to buyers?\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.colname="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>5. How much capital will be required? In answering, please consider:\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Cost of fixed capital:\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Pickup truck\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Stake 1 1/2 ton\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Office equipment:\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Computers\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.r</field><field>esize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style</field><field>="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colnam</field><field>e="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Desk\n\n►<field>entry.resize="same"</field><field>entry.style</field><field>="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Photocopy machine\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] File cabinet and supplies\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Sizing machine\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Chill room\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.align="right"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&b;•\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Miscellaneous\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.nameend="4"</field><field>entry.namest="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Record books and set up--co-op books\n\n\n\n\n\n\n►<field>row.date.updat</field><field>ed="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="s</field><field>ame"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="</field><field>2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Attorney\'s fees\n\n►<field>entry.resize="s</field><field>ame"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Recording fee\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>&ballot;[] Retainer fee (lawyer)\n\n►<field>entry.colname="5"</field><field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.charoff="2"</field><field>entry.colname="3"</field><field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.nameend="3"</field><field>entry.namest="1"</field><field>entry.resize="same"</field><field>entry.style="block"</field>II. Outlets and Proposed Marketing Methods:\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>1. Often it is advisable to confine marketing operations to one trade group, i.e., retailers or wholesalers. Trying to sell to several groups may arouse antagonism from one or other of established business groups. What should the committee suggest in this respect?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>2. What reception should the cooperative expect from potential customers and competitors now in business?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>3. Will the cooperative establish its own place of business with a complete sales and delivery force, or will it employ an agent on a contract basis?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>4. Will the policy be to market a uniform product that can be standardized under supervision of the cooperative management, and labeled and advertised as a cooperative product, or to act as an agent for produce packed on the members\' farms?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>5. What will be price policy on sales?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.nameend="5"</field><field>entry.namest="1"</field><field>entry.resize="same"</field><field>entry.style="block"</field>III. Organization and financing:\n\n\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>1. Where will the place, or places, of business be established?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>2. What should the scope of the cooperative be? How many products will the cooperative attempt to handle at the beginning? Later?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>3. From how big an area will members be accepted? The state? One country? One district?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>4. What should the name of the cooperative be?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>5. Financing: Will the cooperative be incorporated or not? Will it be a stock organization by which capital is raised from the sale of interest-bearing stock certificates, or a nonstock cooperative that requires only membership fees? Usually this is decided by the amount of capital required.\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>6. Membership: Will there be any limitation on the least volume of business that possible members will give to the cooperatives?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>7. How many potential members are there, and how many of them will join a cooperative?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>8. What will be the membership fee?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>9. What plan is proposed to secure maximum membership?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>10. Will the cooperative do business with nonmembers? To what extent?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>11. How will the Board of Directors be chosen? On the basis of size of business, by districts, at large, or a combination? What fees and expenses will be paid to Directors?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>12. Manager: What type of manager will be employed? At what salary? What will his or her duties and authority be in relation to the Board of Directors?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>en</field><field>try.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resi</field><field>ze="same"</field><field>entry.style="block"</field>13. What reports will the manager and Board of Directors give to members? How often?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.nameend="5"</field><field>entry.namest="1"</field><field>entry.resize="same"</field><field>entry.style="block"</field>IV. Marketing Agreement:\n\n\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>1. To establish a new business there must usually be a guarantee of a certain volume of business. How shall member contracts of volume be made? On an estimated quantity per month? Or a percentage of total production?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>2. Meeting the cost of operating a business requires certain deductions from gross returns. Shall deductions be on a fee per unit sold or on a percentage of dollar sales?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>3. How long a period shall the marketing agreement cover?\n\n\n\n\n\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.nameend="5"</field><field>entry.namest="2"</field><field>entry.resize="same"</field><field>entry.style="block"</field>4. What will be the penalties for noncompliance of members? What escape clauses will be allowed members in the contract? What escape clauses for the cooperative?\n\n\n\n\n\n\n\n\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I048UDS"</field>West\'s Key Number Digest, Agriculture &key;6\n1. Preincorporation Agreements and Certificate of Incorporation\n32:15. Preorganization membership agreement\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n►<field>form.du="0"</field><field>form.fid="Ia3931300181a11e68738d3db3a7c8dd3"</field>ORGANIZATION AGREEMENT \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>The undersigned, a producer of agricultural products, referred to as the &ldquo;“Producer,&rdquo;” together with other signers of agreements similar to this agreement, for the purpose of engaging in [list of purposes for organization], propose to organize an agricultural cooperative association without capital stock under the laws of the State of New York, as provided in this agreement. The undersigned agrees, both for the undersigned and for the express benefit of the association to be organized, as follows:\n \n►<field>fi.lvl="1"</field>1. ORGANIZATION COMMITTEE\n \n►<field>fi.lvl="2"</field>(a). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>The association will be organized with a suitable certificate of incorporation and bylaws as determined by an organization committee consisting of the following persons: [names of committee members] and [addresses of committee members].\n►<field>fi.lvl="2"</field>(b). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>This committee may, in the discretion of a majority of its members, increase its membership, fill any committee vacancy, and appoint any committees deemed necessary to conduct the details of its affairs. The committee or any committee designated by it may prescribe an organization fee to be paid by each person signing an organization agreement similar to this document and may incur necessary obligations, make necessary expenditures, and take any such action as may, in its discretion, be deemed advisable to further the organization of the association.\n►<field>fi.lvl="1"</field>2. BYLAWS\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>The bylaws of the association will provide, among other things, that [enumeration of principal proposed provisions].\n►<field>fi.lvl="1"</field>3. ORGANIZATIONAL MEETING\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>If, on or before [date of notice], the organization committee is of the opinion that sufficient sign-up has been obtained to enable the association to function effectively, the committee will, by notice to be published in one or more newspapers of general circulation in the area in which those who sign the agreements like this one reside, specify a date and a place for a meeting of those who sign such agreements to enable those attending such meeting to conclusively determine, by majority vote, if a sufficient sign-up has been obtained to justify the formation and operation of the association, and to consider such other business as may be deemed expedient. Notice of the action there taken will be published in one or more newspapers of general circulation in the area.\n►<field>fi.lvl="1"</field>4. ACCOUNTING OF RECEIPTS AND EXPENDITURES\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>The organization committee will keep full, true, and detailed accounts of all receipts and of all expenditures of every kind; will have the accounts audited; will render a written report of the audit to the board of directors of the association when organized; and will thereupon turn over to the association any balance remaining in its hands free of obligation. If the association is not so organized, the unexpended balance of the account will be prorated to those who contributed to it.\n►<field>fi.lvl="1"</field>5. REVOLVING FUND CERTIFICATES\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>Producer subscribes for [number of revolving fund certificates] revolving fund certificates, each of the face value of $[dollar amount of face value], and agrees to pay for the certificates as follows: [terms of payment].\n►<field>fi.lvl="1"</field>6. APPLICATION FOR MEMBERSHIP; SIGNATURE OF MARKETING AGREEMENT\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>Producer applies for membership in the association when organized and expressly agrees that signature to the marketing agreement will be deemed, to all intents and purposes, the same as signature to this organization agreement, all of which shall be irrevocable, except as provided in Paragraph 3 of this organization agreement and Paragraph [designation of paragraph] of the marketing agreement or the association\'s bylaws, and Producer so agrees in order to induce other producers to sign agreements such as this for Producer\'s benefit as well as for their own general benefit.\n►<field>fi.lvl="1"</field>7. NOTICE OF ACCEPTANCE\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>Acceptance of this agreement will be deemed conclusive on the mailing, by the association, of a notice to that effect to Producer at Producer\'s address noted below, and such mailing and notice will be conclusively established by the affidavit of the secretary of the corporation.\n►<field>fi.lvl="1"</field>8. MARKETING AGREEMENT\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>Subject to the terms of this agreement, Producer agrees to be bound by the terms of the following marketing agreement, which, on its acceptance by the corporation, may be used separately apart from this organization agreement: [recitation of marketing agreement terms].\n ►<field>fal.lo="n"</field>Dated: [date of agreement], at [name of city], New York.\n►<field>fsig.pos="r"</field>►<field>fal.lo="y"</field> ______________ \n[Name of producer]\n ►<field>fal.lo="n"</field>[Address of producer]\n ►<field>fal.lo="n"</field>Accepted: [date of acceptance]\n►<field>fsig.pos="r"</field>►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Names of other parties]\nNotes to Form\n\n \n\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>An alternative to publishing the actions taken at the organization meeting may be to provide that the proposed organization will not be organized unless a certain proportion of the acreage or tonnage within a certain district is signed up. A further provision is then generally made that the certificate of the organization committee to the effect that the required acreage or tonnage has been signed will be conclusive.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>At the first meeting of the corporation, bylaws are required to be adopted, by a majority vote of the members or by their written assent, to regulate the conduct and management of the corporation. ►<field>st.ref.id="I05QQYN"</field>BCL &s;§601(a).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>The terms and conditions of membership in an agricultural corporation must be prescribed in the bylaws. ►<field>st.ref.id="I05QQSZ"</field>Coop. Corp. Law &s;§111(a).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>The term &ldquo;“agricultural products&rdquo;” includes horticultural, viticultural, dairy, livestock, poultry, bee, and any farm products. ►<field>st.ref.id="I05NXO3"</field>Coop. Corp. Law &s;§111(b).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>Any five or more producers of agricultural products may form a cooperative agricultural corporation, either with or without capital stock. ►<field>st.ref.id="I05L4J7"</field>Coop. Corp. Law &s;§112.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|15|32:15"</field>For other forms of subscription agreements, see ►<field>x.ref.id="I05IBEB"</field>&ss;§§17:1 et seq.\nResearch References\n ►<field>tk.ref.id="I05FI9F"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05QQSX"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I05NXO1"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I05L4J5"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I05IBE9"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I05FI9D"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I048UKY"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n32:16. Revolving fund certificate\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n►<field>form.du="0"</field><field>form.fid="Ia3933a10181a11e68738d3db3a7c8dd3"</field>REVOLVING FUND CERTIFICATE SERIES \n ►<field>fcap.ref="manual"</field>►<field>tbl.date.updated="0"</field><field>tbl.orient="port"</field><field>tbl.position="l"</field><field>tbl.table.width="100"</field><field>tbl.uuid="I034JMZ"</field>\n►<field>table.resize="same"</field><field>table.tabstyle="text"</field>\n►<field>tgroup.cols="2"</field>\n►<field>colspec.colname="1"</field><field>colspec.colnum="1"</field><field>colspec.colwidth="1*"</field>\n►<field>colspec.colname="2"</field><field>colspec.colnum="2"</field><field>colspec.colwidth="19*"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>No. \n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>[Number of certificate]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>[Date of certificate]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="sa</field><field>me"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>$[Dollar amount of certificate]\n\n\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|16|32:16"</field>This certifies that [name of member], of [address of member], [name of city], [name of county], [name of state], has furnished $[dollar amount of capital] to this corporation as capital subject to the following conditions:\n \n►<field>fi.lvl="1"</field>1. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|16|32:16"</field>This and other revolving fund certificates of the same series may be retired in the sole discretion of the board of directors, either fully or on a pro rata basis, but certificates issued in prior years will be entitled to priority (except as provided in this document) in retirement.\n►<field>fi.lvl="1"</field>2. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|16|32:16"</field>The amount stated in this certificate will bear only such rate of interest, if any, as the board of directors may fix, from time to time, in no event to exceed [percentage rate of annual interest]&percnt;% per year.\n►<field>fi.lvl="1"</field>3. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|16|32:16"</field>This certificate is transferable only on the books of the corporation.\n►<field>fi.lvl="1"</field>4. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|16|32:16"</field>This and other certificates are subject in all respects to the corporation\'s bylaws and are junior and subordinate to all debts of the corporation, both secured and unsecured.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|16|32:16"</field>Upon the winding up or liquidation of the corporation in any manner, after full payment of all of its creditors, all revolving fund certificates will then be retired in full or on a pro rata basis, without priority.\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of certificate]\n ►<field>fsig.pos="r"</field>►<field>fal.lo="n"</field>[Name of corporation]\n ►<field>fal.lo="n"</field>By:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of authorized representative]\n ►<field>fal.lo="n"</field>[Title of authorized representative]\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Attested:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of secretary]\n ►<field>fal.lo="n"</field>Secretary\n ►<field>fal.lo="n"</field>[Corporate seal]\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I048UM5"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I048UM8"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I04BNR4"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I04EGW0"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I04HA0W"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I048UMA"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I048URY"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n32:17. Certificate of incorporation-- Agricultural cooperative marketing corporation\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia393d650181a11e68738d3db3a7c8dd3"</field>\n►<field>fi.lvl="1"</field>CERTIFICATE OF INCORPORATION OF [NAME OF CORPORATION]\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>We, the undersigned, [names of persons engaged in production of agricultural products], have voluntarily associated ourselves together for the purpose of forming a nonprofit agricultural cooperative corporation under and by virtue of the laws of the State of New York and particularly under the provisions of Chapter 77, Article 6 of the Consolidated Laws of the State of New York and certify to the following:\n \n►<field>fi.lvl="2"</field>ARTICLE I  NAME\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The name of this corporation shall be [name of corporation].\n►<field>fi.lvl="2"</field>ARTICLE II  PURPOSE\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The purpose for which the corporation is formed is [description of purpose for incorporation].\n►<field>fi.lvl="2"</field>ARTICLE III  TERRITORY AND PRINCIPAL OFFICE\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The territory in which the corporation\'s operations are to be conducted is [description of territory]; the principal office for the transaction of business of the corporation within the State of New York is located at [address of corporation], [name of city], [name of county]. [Name of agent], whose address is [[business address of agent]/[residential address of agent]], [name of city], [name of county], [name of state], is the agent in charge of this corporation and the person upon whom process against the corporation may be served.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The Secretary of State is also designated as an agent of this corporation for the purpose of accepting process, which may be served against the corporation, and the Secretary of State may mail any process that has been served upon such official against the corporation to [address of corporation], [name of city], [name of county], [name of state].\n►<field>fi.lvl="2"</field>ARTICLE IV  DURATION\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The term of existence is [perpetual/[specification of time period]/until [date of expiration]].\n►<field>fi.lvl="2"</field>ARTICLE V  NAMES AND ADDRESSES OF INCORPORATORS\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The names and post office addresses of the incorporators are:\n ►<field>tbl.date.updated="0"</field><field>tbl.ident="I03BYJU"</field><field>tbl.orient="port"</field><field>tbl.position="l"</field><field>tbl.table.width="100"</field><field>tbl.uuid="I021QFL"</field>\n►<field>table.resize="same"</field><field>table.tabstyle="text"</field>\n►<field>tgroup.align="left"</field><field>tgroup.cols="2"</field>\n►<field>colspec.colname="1"</field><field>colspec.colnum="1"</field>\n►<field>colspec.colname="2"</field><field>colspec.colnum="2"</field>\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Names of Incorporator\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Addresses of Incorporator\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 1]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 1]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 2]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 2]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 3]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 3]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 4]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 4]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 5]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 5]\n\n\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>All of the subscribers are of full age, and at least two-thirds of them are citizens of the United States. At least one of the subscribers is a resident of the State of New York. At least one of the persons named as a director in this certificate is a citizen of the United States and a resident of New York.\n►<field>fvs.amt="1"</field>\n►<field>sel.p="fl"</field><field>sel.tp="alt"</field>[EITHER:]\n \n ►<field>fi.lvl="2"</field>ARTICLE VI  MEMBERSHIP INTERESTS\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The property rights and interests of each member shall be [equal/unequal]. [OPTIONAL: [List of general rules to all members by which property rights and interests, respectively, of each member are to be determined and fixed].]\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>New members shall be admitted on the following terms and conditions: [specification of terms and conditions].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The provisions of this Article shall not be altered, amended, or repealed, except by [written consent/vote of three-fourths of the members of this corporation].[OR:]\n \n ►<field>fi.lvl="2"</field>ARTICLE VI  CAPITAL STOCK\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The corporation shall have capital stock consisting of [number of shares] shares, all having a par value of $[dollar amount of par value]. The capital stock of the corporation shall consist of only one class of shares.[END OF CHOICES]\n►<field>fi.lvl="2"</field>ARTICLE VII  DATE OF ANNUAL MEETING\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The fiscal year of the corporation will commence on [begin date of fiscal year] and end on [end date of fiscal year] following, and the annual meeting of the corporation will be held at its principal office at [time of meeting] on the [ordinal number of day] day of [name of month] of each year.\n►<field>fi.lvl="2"</field>ARTICLE VIII  DIRECTORS\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The number of directors of the corporation shall [be [number of directors]/not be less than [minimum number of directors] or more than [maximum number of directors] as the bylaws may from time to time provide, but in no case less than five].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The names and post office addresses of the directors until the first meeting are:\n ►<field>tbl.date.updated="0"</field><field>tbl.ident="I00ZERK"</field><field>tbl.orient="port"</field><field>tbl.position="l"</field><field>tbl.table.width="100"</field><field>tbl.uuid="I033O6A"</field>\n►<field>table.resize="same"</field><field>table.tabstyle="text"</field>\n►<field>tgroup.align="left"</field><field>tgroup.cols="2"</field>\n►<field>colspec.colname="1"</field><field>colspec.colnum="1"</field>\n►<field>colspec.colname="2"</field><field>colspec.colnum="2"</field>\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Names of Director\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>Addresses of Director\n\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 1]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 1]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 2]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 2]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 3]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 3]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 4]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 4]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[Name 5]\n\n►<field>entry.align="center"</field><field>entry.resize="same"</field><field>entry.style="block"</field>[address 5]\n\n\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>Each director is to serve until the election and qualification of [his/her] successor.\n►<field>fi.lvl="2"</field>ARTICLE IX  MISCELLANEOUS\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>[Recitation of other provisions].\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of certificate of incorporation]\n►<field>fsig.pos="r"</field>►<field>fal.lo="y"</field> ______________ \n\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Names of incorporators]\n ►<field>fal.lo="n"</field>[Acknowledgment]\nNotes to Form\n\nTax Notes \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>Unlike other nonspecific types of cooperative organizations, an agricultural cooperative is defined with precision by the Code, regulations, rulings, and cases. Two kinds of agricultural co-ops qualify as &ldquo;“tax-exempt.&rdquo;” The term is placed in quotes for a reason. While these organizations are stated to be &ldquo;“exempt from taxation&rdquo;” under ►<field>st.ref.id="I05QQES"</field>26 U.S.C.A. &ss;§§521, farmers\' cooperatives are still required to be pay taxes specified in ►<field>st.ref.id="I05NX9W"</field>26 U.S.C.A. &s;§ 1381 et seq.,26 U.S.C.A. &s;§ 1381 then requires them to pay the ordinary income tax and capital gains tax levied by. 26 U.S.C.A. &s;§&s;§ 11 and 1201. Section 1381 then goes on to say that in spite of these taxes being levied against agricultural cooperatives, they are considered to be exempt from income taxes for any purpose in which the law refers to exempt organizations.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The two types of agricultural cooperatives that are recognized are the marketing co-op and the purchasing co-op. The requirements for these organizations are found at ►<field>st.ref.id="I05L450"</field>26 U.S.C.A. &s;§521(b)(1), (4).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The result is that an agricultural cooperative is entitled to a special deduction for patronage dividends, dividends on capital stock, and certain other distributions. Otherwise, they pay tax under the regular rules that relate to ordinary income and capital gain. They are not permitted the dividends-received deduction and may not use the consolidated return provisions. ►<field>st.ref.id="I05FHV8"</field>26 C.F.R. &s;§1.522-1(a).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>For a discussion of considerations relevant to the drafting of certificates of incorporation, see ►<field>x.ref.id="I05QQEP"</field>&s;§32:9.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>For a discussion of the formal requirements of certificates of incorporation, see ►<field>x.ref.id="I05NX9T"</field>&ss;§§32:10, 32:12.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>In Article II, the description of the purpose for incorporation could include: to engage in any cooperative activity in connection with the producing, marketing, or selling of agricultural products, as that term is defined in ►<field>st.ref.id="I05L44X"</field>N.Y Cooperative Corporations Law &s;§111(b), and in general to function as an agricultural cooperative corporation within the meaning of ►<field>st.ref.id="I05IB01"</field>N.Y Cooperative Corporations Law &s;§111(a), to which end the return on the stock or membership capital of the corporation shall be limited to 12&percnt;% per year and the value of business done with nonmembers in any fiscal year shall not exceed the value of business done with members during such fiscal year.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The certificate of incorporation must include the purposes of the corporation, which are required to be consistent with the New York Cooperative Corporations Law. ►<field>st.ref.id="I05FHV5"</field>Coop. Corp. Law &s;§11(2).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>For the purposes for which a general cooperative corporation may be formed, see ►<field>st.ref.id="I05QQEN"</field>Coop. Corp. Law &s;§13; for the more specific purposes of an agricultural cooperative corporation, see ►<field>st.ref.id="I05NX9R"</field>Coop. Corp. Law &s;§110.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>Any five or more producers of agricultural products may form a cooperative agricultural corporation, either with or without capital stock, by subscribing and filing a certificate of incorporation. ►<field>st.ref.id="I05L44V"</field>Coop. Corp. Law &s;§112. The certificate of incorporation must be acknowledged by all the incorporators. ►<field>st.ref.id="I05IAZZ"</field>Coop. Corp. Law &s;§11.\nResearch References\n ►<field>tk.ref.id="I05FHV3"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05QQEK"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I05NX9O"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I05QQDF"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I05NX8J"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I05L43N"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I05IAYR"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n ►<field>rc.ref.id="I05FHTV"</field>N.Y. Jur. 2d, Business Relationships &s;§73\nARTICLE I. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>A certificate of incorporation is required to set forth the name of the corporation, which must include the word &ldquo;“Cooperative.&rdquo;” ►<field>st.ref.id="I048V0G"</field>Coop. Corp. Law &s;§11(1).\nResearch References\n ►<field>rc.ref.id="I04BO5C"</field>N.Y. Jur. 2d, Business Relationships &ss;§§164 to 179\nARTICLE II. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The certificate of incorporation is required to set forth the purposes for which the corporation has been formed. ►<field>st.ref.id="I04EHA8"</field>Coop. Corp. Law &s;§11(2).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The powers of cooperative corporations are delineated in ►<field>st.ref.id="I04HAF4"</field>Coop. Corp. Law &s;§14. If desired, these powers may be incorporated into the purpose section of the certificate of incorporation.\nResearch References\n ►<field>rc.ref.id="I048V0I"</field>N.Y. Jur. 2d, Business Relationships &s;§61\nARTICLE III. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The certificate of incorporation is required to designate the Secretary of State as an agent of the corporation for the purpose of receiving process that may be served on the corporation. ►<field>st.ref.id="I04BO5E"</field>Coop. Corp. Law &s;§11(10). If the corporation is to have a registered agent, the certificate of incorporation must include the agent\'s name and address within New York and a statement that he or she is authorized to receive process served against the corporation. ►<field>st.ref.id="I04EHAA"</field>Coop. Corp. Law &s;§11(11).\nResearch References\n ►<field>rc.ref.id="I04HAF6"</field>N.Y. Jur. 2d, Business Relationships &s;§73\n ►<field>rc.ref.id="I04K3K2"</field>N.Y. Jur. 2d, Business Relationships &ss;§§1062, 1063\nARTICLE IV. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>The certificate of incorporation must state the term for which the corporation is to exist. ►<field>st.ref.id="I048V66"</field>Coop. Corp. Law &s;§11(3).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>Subject to any limitations provided by statute or in a certificate of incorporation, a corporation has the power, in furtherance of its corporate purposes, to have perpetual duration. ►<field>st.ref.id="I04BOB2"</field>BCL &s;§202(a)(1).\nResearch References\n ►<field>rc.ref.id="I048V68"</field>N.Y. Jur. 2d, Business Relationships &s;§110\nARTICLE V. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>A certificate of incorporation is required to set forth the names and post office addresses of the incorporators. ►<field>st.ref.id="I04BOB4"</field>Coop. Corp. Law &s;§11(5).\nARTICLE VI. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|17|32:17"</field>If the corporation is organized with capital stock, the certificate of incorporation must state the total amount of such stock, the total number, if any, of the shares without par value, and the total number and par value of any shares having a par value, and, if the shares are to be classified, the number of shares to be included in each class and all of the designations, preferences, privileges, and voting rights or restrictions and qualifications of the shares of each class. ►<field>st.ref.id="I04EHG0"</field>Coop. Corp. Law &s;§11(8).\n32:18. Certificate of incorporation-- Provision-- Purpose-- Dairy cooperative corporation\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia393fd60181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|18|32:18"</field>The purpose of this organization is to operate on a nonprofit cooperative basis for the mutual benefit of its member producers of milk, milk products, or any other dairy products and to engage in any drying, processing, canning, packing, handling, storing, or utilizing of any dairy products of its members or the manufacturing or marketing of by-products.\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I048V6A"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04BOB6"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I04EHG2"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I04HAKY"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I048V6D"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I04BOB9"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I04EHG5"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n ►<field>rc.ref.id="I04HAL1"</field>N.Y. Jur. 2d, Business Relationships &s;§62\n ►<field>rc.ref.id="I04K3PX"</field>Am. Jur. 2d, Cooperative Associations &s;§4\n32:19. Certificate of incorporation-- Provision-- Purpose-- Growers\' association\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3942470181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|19|32:19"</field>The purposes for which this organization is formed are:\n \n►<field>fi.lvl="1"</field>(1). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|19|32:19"</field>to associate its members together for their mutual benefit as growers of [description of product];\n►<field>fi.lvl="1"</field>(2). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|19|32:19"</field>to assist its members without profit to the Corporation in promoting the demand for and sale of [description of product 1] and [description of product 2] through advertising and promotional work with processors, handlers, consumers, and others;\n►<field>fi.lvl="1"</field>(3). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|19|32:19"</field>to provide its members with marketing information to represent its members\' point of view to state and federal agencies, to buyers, and to the public;\n►<field>fi.lvl="1"</field>(4). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|19|32:19"</field>to cooperate with similar associations or agencies in performing any of the purposes for which that association is formed; and\n►<field>fi.lvl="1"</field>(5). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|19|32:19"</field>to perform any and all things not inconsistent with law that affect the mutual interests of [description of product] growers.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|19|32:19"</field>If the principal activities of a cooperative agricultural corporation are connected with the marketing, processing, manufacture, sale, or other dispositions of agricultural products, it may be termed a cooperative marketing corporation and incorporated as such. ►<field>st.ref.id="I048V7L"</field>Coop. Corp. Law &s;§112.\nResearch References\n ►<field>tk.ref.id="I04BOCH"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04EHHD"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I048VD8"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I04BOI4"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I04EHN0"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I04HARW"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I04K3WS"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n ►<field>rc.ref.id="I048VDB"</field>N.Y. Jur. 2d, Business Relationships &s;§61\n32:20. Certificate of incorporation-- Provision-- Amount of capital\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3947290181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|20|32:20"</field>The amount of capital with which this corporation will begin business is $[dollar amount of capital].\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|20|32:20"</field>For further listings of matters that may be included in certificates of incorporation, see ►<field>x.ref.id="I05QQ0F"</field>&ss;§§17:1 et seq.\nResearch References\n ►<field>tk.ref.id="I05NWVJ"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05L3QN"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I05IALR"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I05QQ0D"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I05NWVH"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I05L3QL"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I05IALP"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n ►<field>rc.ref.id="I05FHGT"</field>N.Y. Jur. 2d, Business Relationships &s;§73\n32:21. Certificate of incorporation-- Provision-- Several classes of shares; distinction\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39499a0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|21|32:21"</field>The shares of the corporation shall consist of [total number of shares] shares divided into [number of shares] shares of [specification of stock] stock of the par value of $[dollar amount of par value] per share, and [number of shares] shares of [specification of stock] stock of the par value of $[dollar amount of par value] per share.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|21|32:21"</field>The board of directors may, subject to the approval of a majority of the members, fix the dividend rate, the conversion rights, the voting rights, the redemption price, and the liquidation preferences of any wholly unissued class of shares, or all or any of them.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|21|32:21"</field>The shares of preferred stock may be reduced at such redemption price and bear such particular designation as the board of directors shall by resolution determine and fix prior to the issuance of such shares.\nNotes to Form\n\nTax Notes \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|21|32:21"</field>Farmers\' cooperatives need not issue capital stock but are not penalized for doing so. ►<field>st.ref.id="I05QPZ7"</field>26 U.S.C.A. &s;§521(b)(2). Dividends on the stock must, however, be limited to the legal rate of interest or 8&percnt;%, whichever is greater. In addition, substantially all the stock must be owned by &ldquo;“producers who market their products or purchase their supplies and equipment&rdquo;” through the cooperative. Distributions on the stock are deductible by the cooperative against its income.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|21|32:21"</field>If the corporation is organized with stock, the certificate must state the total amount of such stock, the total number, if any, of the shares without par value, and the total number and par value of any shares having a par value. If the shares are to be classified, it must also state the number of shares to be included in each class and all of the designations, preferences, privileges, and voting rights or restrictions and qualifications of the shares of each class. ►<field>st.ref.id="I05NWUB"</field>Coop. Corp. Law &s;§11(8).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|21|32:21"</field>Dividends payable on all classes of preferred stock may not exceed 12&percnt;% per year. ►<field>st.ref.id="I05QPZ5"</field>Coop. Corp. Law &s;§111(a).\nResearch References\n ►<field>tk.ref.id="I05NWU9"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05QPZ2"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I05NWU6"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I05L3PA"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I05IAKE"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I05FHFI"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I05QPTF"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n ►<field>rc.ref.id="I05NWOJ"</field>N.Y. Jur. 2d, Business Relationships &s;§30\n32:22. Certificate of incorporation-- Provision-- Limitation of personal liability of board of directors\nCLS\nCOL\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia394e7c0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>No director of this cooperative shall be personally liable to this cooperative or its members for monetary damages for breach of fiduciary duty as a director, except for liability:\n \n►<field>fi.lvl="1"</field>(a). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>For a breach of the director\'s duty of loyalty to this cooperative or its members;\n►<field>fi.lvl="1"</field>(b). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>For acts or omissions not in good faith or that involve intentional misconduct or a knowing violation of law;\n►<field>fi.lvl="1"</field>(c). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>For a transaction from which the director derived an improper personal benefit; or\n►<field>fi.lvl="1"</field>(d). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>For an act or omission occurring prior to the date when the provisions of this Article (or predecessor thereto) became effective.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>It is the intention of the members of this cooperative to eliminate or limit the personal liability of the directors of this cooperative to the greatest extent permitted under ►<field>st.ref.id="I048VKG"</field>Section 402(b) of the New York Business Corporation Law. If amendments to that section or other law are passed after the effective date of this Article, which authorize cooperatives to act to further limit or eliminate the personal liability of directors, then the liability of the directors of this cooperative shall be limited or eliminated to the greatest extent permitted by such statute as amended. Any repeal or modification of this Article by the members of this cooperative shall not adversely affect any right of or any protection available to a director of this cooperative that is in existence at the time of such repeal or modification.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>The certificate of incorporation may set forth a provision eliminating or limiting the personal liability of directors to the corporation or its shareholders for damages for any breach of duty in such capacity, provided that no such provision shall eliminate or limit:\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>(1) the liability of any director if a judgment or other final adjudication adverse to him or her establishes that his or her acts or omissions were in bad faith or involved intentional misconduct or a knowing violation of law or that he or she personally gained in fact a financial profit or other advantage to which he or she was not legally entitled or that his or her acts violated Section 719 of the Business Corporation Law; or\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>(2) the liability of any director for any act or omission prior to the adoption of a provision authorized by this paragraph. <field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|22|32:22"</field>See ►<field>st.ref.id="I048VKI"</field>BCL &s;§402(b).\nResearch References\n ►<field>tk.ref.id="I048VKL"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04BOPH"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;920\n ►<field>tk.ref.id="I04EHUD"</field>West\'s Key Number Digest, Antitrust and Trade Regulation &key;921\n ►<field>tk.ref.id="I04HAZ9"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3513\n ►<field>tk.ref.id="I048VLQ"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3515\n ►<field>tk.ref.id="I04BOQM"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3527\n ►<field>tk.ref.id="I04EHVI"</field>West\'s Key Number Digest, Corporations and Business Organizations &key;3533 to 3535\n ►<field>rc.ref.id="I04HB0E"</field>N.Y. Jur. 2d, Business Relationships &s;§70\n2. Bylaws\n32:23. Bylaws of agricultural marketing cooperative-- Nonstock\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia396bc80181a11e68738d3db3a7c8dd3"</field>\n►<field>fi.lvl="1"</field>BYLAWS OF [NAME OF CORPORATION]\n \n►<field>fi.lvl="2"</field>ARTICLE I  NAME AND HEADQUARTERS\n \n►<field>fi.lvl="3"</field>Section 1. Name. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The name of the cooperative shall be [name of cooperative], referred to in these bylaws as &ldquo;“Cooperative.&rdquo;”\n►<field>fi.lvl="3"</field>Section 2. Headquarters. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The headquarters of the cooperative shall be located at [address of cooperative], [name of city], [name of county], State of New York.\n►<field>fi.lvl="2"</field>ARTICLE II  FUNCTIONS AND OBJECTIVES\n \n►<field>fi.lvl="3"</field>Section 1. Business. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The business to be carried on by the cooperative shall be [description of business].\n►<field>fi.lvl="3"</field>Section 2. Other Objectives. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Other objectives of the cooperative shall be:\n \n►<field>fi.lvl="4"</field>(a). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>cooperative service for the benefit of the members and of the consuming public;\n►<field>fi.lvl="4"</field>(b). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>affiliation or cooperation with other cooperative organizations of a like nature in the local community, the state, the nation and other countries;\n►<field>fi.lvl="4"</field>(c). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>promotion of the cooperative ideal through educational activities; and\n►<field>fi.lvl="4"</field>(d). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>such other objectives as may be set out in the certificate of incorporation.\n►<field>fi.lvl="2"</field>ARTICLE III  MEMBERSHIP\n \n►<field>fi.lvl="3"</field>Section 1. Eligibility. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any person, irrespective of age, sex, race, nationality, political opinion, or religious belief, and any firm, partnership, or corporation, including both landlords and tenants in share tenancies, who is engaged in the production of [name of agricultural product] in the territory in which the corporation is engaged in business, who agrees to be a patron of the corporation, and who pays such membership fees and meets such other conditions as may be prescribed by the board of directors is eligible to become a member of this cooperative.\n►<field>fi.lvl="3"</field>Section 2. Number. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The number of members of the corporation shall be limited to [number of members].\n►<field>fi.lvl="3"</field>Section 3. Fees, operating, and other charges. \n \n►<field>fi.lvl="4"</field>a. Fee Required. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Each person on becoming a member of the corporation will pay to the corporation a fee of $[dollar amount of fee], of which amount $[dollar amount of annual subscription] shall be in payment of [number of years] year\'s subscription to [[name of publication]/[name of other similar publication of cooperative]] and the balance of which shall be applied to [specification of intended use of balance amount].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The corporation\'s operating expenses shall be met by a percentage charged on returns for products sold and by a percentage charged on supplies purchased, the amount of such charges to be fixed by the board of directors. Payment for the expenses will be made on the first day of each month.\n►<field>fi.lvl="4"</field>b. Payment of Fee and Membership Shares. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The fee must be paid in cash. A membership share may be paid for by:\n \n►<field>fi.lvl="5"</field>(1). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Payments in lawful money of the United States;\n►<field>fi.lvl="5"</field>(2). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Application of patronage refund;\n►<field>fi.lvl="5"</field>(3). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Credit of interest of other debts due; or\n►<field>fi.lvl="5"</field>(4). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any combination of the above.\n►<field>fi.lvl="3"</field>Section 4. Property Rights of Members. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The corporation will not have any capital stock. The following membership rights, privileges and restrictions shall apply:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>A member shall be entitled to only one vote and that only after the member has purchased one membership share. The full $[dollar amount of fee] must be paid (see Subsection 3b above) within a period of [number of months] months from the date of application. On failure to make such timely payment, the amount of the member\'s credit shall, at the discretion of the board of directors and after [number of days] days\' notice by [registered mail/[specification of mode of delivery]] to the member at the member\'s last-known address, be refunded and the membership cancelled. If a member cannot be located, the amount shall be applied to the work of [name of committee].\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Each member\'s patronage refund and share dividends shall be credited to the member as payment on membership shares until the member owns [number of shares] shares, and thereafter the member\'s patronage refunds and share dividends, if their combined total is $[dollar amount of combined total] or more, shall be paid in cash unless the member gives other directions, provided, however, that the patronage refund shall be paid in the manner and to the extent required by the Internal Revenue Code and Treasury Department Regulations to qualify for income tax deductions or exclusions for this cooperative. Patronage refunds and share dividends not paid out shall be applied toward purchase of additional shares.\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Dividends shall be paid on membership shares as provided below and shall accrue from the first day of the first full month during which a member owns at least [number of membership shares] membership shares. For all purposes relating to the payment of dividends, all shares purchased by a member not later than the [ordinal number of day] day of any month shall be deemed to have been owned by the member on the first day of that month.\n►<field>fi.lvl="4"</field>d. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>A member may not hold more than [percentage of total membership shares]&percnt;% of the total of all membership shares issued and outstanding at any one time.\n►<field>fi.lvl="4"</field>e. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Membership shares are nontransferable except within a member\'s family and then only on approval of the board of directors after written request for such transfer.\n►<field>fi.lvl="4"</field>f. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The board of directors may, after written notice by first class mail, recall, at par value, the membership shares and terminate the membership of any member who has failed to patronize or participate in the organizational activities of the cooperative for at least one year.\n►<field>fi.lvl="4"</field>g. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>For the purpose of this paragraph, &ldquo;“family&rdquo;” shall be defined as those individuals living in one household as the single purchasing unit. Individuals wishing to be grouped in such purchasing unit must so inform the cooperative in writing.\n►<field>fi.lvl="4"</field>h. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Each person who becomes or remains a member of this cooperative after the adoption of this bylaw (ARTICLE III, Section 4h) consents to include in [his/her] gross income for federal income tax purposes the amount of any patronage refund paid to him or her by this cooperative by written notice of allocation (as defined in the Internal Revenue Code), except to the extent that such a patronage refund is not income to the member because: (1) it is attributable to the purchase of personal, living, or family items; or (2) it should be properly treated as an adjustment to the tax basis of property previously purchased. The term &ldquo;“patronage refund&rdquo;” as used herein shall have the same meaning as that term is used in the Internal Revenue Code.\n►<field>fi.lvl="3"</field>Section 5. Liability for Debts of Cooperative. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Membership shares are nonassessable. A member of this cooperative shall not be liable for the debts of the corporation to an amount exceeding the sum remaining unpaid on the membership fee or subscription to the capital stock, including any unpaid balance on any promissory notes given in payment.\n►<field>fi.lvl="3"</field>Section 6. Withdrawal of Amounts Credited on Purchase of Membership Shares. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>A member may withdraw, partially or wholly, amounts credited toward the purchase of membership shares, subject to the following provisions:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Member must submit written notice to the cooperative of intent to withdraw share credits.\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Complete withdrawal necessitates resignation from membership.\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Resignation from membership is effective when approved by the board of directors.\n►<field>fi.lvl="4"</field>d. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Partial withdrawal is permitted only if at least [number of membership shares to be retained] membership shares are retained.\n►<field>fi.lvl="4"</field>e. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Credits shall be reduced proportionately if the total membership share credits of all members are impaired.\n►<field>fi.lvl="4"</field>f. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Member agrees to accept payment in goods or services purchased from the cooperative or in cash at the election of the cooperative.\n►<field>fi.lvl="4"</field>g. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Payment shall be made within one year from date of receipt of notice of intent, providing such payment will not, in the opinion of the board of directors, unduly impair the finances of the cooperative.\n►<field>fi.lvl="3"</field>Section 7. Suspension or Termination. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>If, following a hearing, the board of directors finds that a member has ceased to be an eligible member; has not, for a period of [number of years] years, marketed through the corporation the products covered by the marketing agreement or agreements with the corporation or has not otherwise patronized the corporation; has moved out of the territory in which the corporation is operating; or has died or has withdrawn or resigned from the corporation, the board may suspend such member\'s rights as a member or terminate the membership. On termination of membership in the corporation, the membership certificate will be cancelled, all rights and interests of such member in the corporation will cease, and the member will be entitled only to payment or credit for the equitable appraised value of the member\'s property rights and interests in the corporation, as conclusively determined by the board of directors. In determining property rights and interests, all amounts allocated to each member or evidenced by certificates of any kind will be excluded, and any such amounts will be accounted for to members in accordance with the applicable terms and conditions. No action taken under this section will impair the obligations or liabilities of either party under any contract with the corporation, which may be terminated only as provided in these bylaws.\n►<field>fi.lvl="3"</field>Section 8. Method of Resignation. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>After a person has been a member of the corporation for [number of years] years, the member may withdraw or resign from the corporation in any year on the last day of the anniversary month in which such member was accepted as a member by notifying the corporation of the intention to withdraw or resign, such notice to be given between the [ordinal number of day] and the [ordinal number of day] of the month immediately prior to the effective date of withdrawal or resignation. If the member fails to withdraw or resign in any year, as stated above, it is agreed that such failure will constitute conclusive evidence that the member has renewed membership in the corporation for another year.\n►<field>fi.lvl="2"</field>ARTICLE IV  BUSINESS PRINCIPLES AND PRACTICES\n \n►<field>fi.lvl="3"</field>Section 1. Governing Principles. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The business of this cooperative shall be conducted according to the following principles:\n \n►<field>fi.lvl="4"</field>a. Democratic control. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>There shall be one vote for each member. Voting by proxy is not permitted.\n►<field>fi.lvl="4"</field>b. Open membership. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Membership is open to all producers of [name of agricultural product] acting in good faith without regard to age, sex, race, nationality, political opinion, or religious belief.\n►<field>fi.lvl="4"</field>c. Limited dividends on capital. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>In accordance with New York laws governing cooperative corporations, dividends on membership shares, if earned and declared, shall not exceed [percentage of maximum dividends on membership shares]&percnt;% per year.\n►<field>fi.lvl="4"</field>d. Cash sales at prevailing prices. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Goods and services shall be supplied to patrons at prevailing prices of the neighborhood. All sales shall be on a cash basis.\n►<field>fi.lvl="4"</field>e. Patronage refunds based on purchases. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The distribution of all patronage refunds shall be on the basis of amount of purchases.\n►<field>fi.lvl="4"</field>f. Education and promotion. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Funds shall be provided for education and promotion.\n►<field>fi.lvl="4"</field>g. Neutrality. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The cooperative shall be neutral in religion and politics.\n►<field>fi.lvl="4"</field>h. Business with nonmembers. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>In any fiscal year, the value of business done with nonmembers shall not exceed the value of business done with members during the same period.\n►<field>fi.lvl="3"</field>Section 2. Fiscal Year. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The fiscal year shall end on the [day of week] nearest to the [ordinal number of day] day of [name of month]. The succeeding fiscal year shall begin on the following [day of week].\n►<field>fi.lvl="3"</field>Section 3. Net Surplus Savings and Earnings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>At the end of each fiscal year, the net surplus savings and earnings of the cooperative (referred to as net earnings) shall be determined and applied as follows:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Capital shall be restored to the extent of any impairment thereof.\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>An amount equal to [percentage of net earnings]&percnt;% of net earnings shall be set aside as a reserve fund until such fund shall equal an amount representing [percentage of total membership shares outstanding]&percnt;% of the total membership shares outstanding.\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Dividends at the rate of [percentage rate of dividends]&percnt;% per annum shall be paid on membership certificates to all members who own at least [number of membership certificates] membership certificates, provided, however, that not more than [percentage of net earnings]&percnt;% of the net earnings shall be paid out as dividends on membership shares. In the event [percentage of net earnings]&percnt;% of the net earnings is insufficient to pay dividends as provided above at the rate of [percentage rate of dividends]&percnt;%, the rate shall be determined by dividing a sum equal to [percentage of net earnings]&percnt;% of the net earnings by the aggregate face value of all certificates on which dividends are payable under these bylaws.\n►<field>fi.lvl="4"</field>d. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Patronage refunds shall be paid as follows:\n \n►<field>fi.lvl="5"</field>(1). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Patronage refunds shall be paid on recorded sales. The board of directors shall, from time to time, provide the method and procedure by which business transacted with the cooperative by its patrons shall be recorded by the cooperative. Each patron shall be responsible for meeting all requirements as established pertaining to participation in keeping such patronage records. The sales and other business transactions so entered in the patronage records of the cooperative shall constitute recorded sales.\n►<field>fi.lvl="5"</field>(2). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The amount available for payment of such refunds shall be: (a) that proportion of net earnings which the &ldquo;“recorded sales&rdquo;” of the cooperative bears to the total sales of the cooperative for the fiscal year; or (b) total net earnings for the fiscal year, less the amount paid to the restoration of capital, the reserve fund, and the payment of dividends on membership certificates, as required by law or by these bylaws, whichever is the smaller.\n►<field>fi.lvl="5"</field>(3). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The rate or percentage of such refunds shall be determined by dividing the amount available for payment of refunds by the recorded sales of the cooperative for the fiscal year.\n►<field>fi.lvl="5"</field>(4). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>On written authorization from a member, $[dollar amount of sum payable] of the sum payable to such member as dividends or as a patronage refund at the end of any fiscal year may be retained by the cooperative in payment of a year\'s subscription to &ldquo;“[name of publication]&rdquo;” or other similar publication of the cooperative.\n►<field>fi.lvl="3"</field>Section 4. Computation of Net Earnings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>For the purpose of this article, the net earnings of the cooperative shall be computed:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Before dividends on shares;\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>After provisions for federal income tax; and\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>After allocating to the reserve fund the net amount, after federal income taxes, of any gains or losses resulting from the sale or exchange of land, buildings, or equipment, provided the net gain or loss from all such sales or exchanges exceeds $[dollar amount of minimum net gain or loss] for the taxable year.\n►<field>fi.lvl="2"</field>ARTICLE V  GOVERNMENT AND MANAGEMENT\n \n►<field>fi.lvl="3"</field>Section 1. Membership Control. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The control of the cooperative shall be vested in the membership.\n►<field>fi.lvl="3"</field>Section 2. Function of Membership Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The function of the membership meetings shall be primarily:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To consider legislation.\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To conduct elections.\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To discuss policies and problems of administration and management.\n►<field>fi.lvl="4"</field>d. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To make recommendations to the board of directors.\n►<field>fi.lvl="4"</field>e. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To consider educational programs.\n►<field>fi.lvl="3"</field>Section 3. Administration by Board of Directors. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The administration of the cooperative is vested in a board of directors responsible to the membership.\n►<field>fi.lvl="3"</field>Section 4. Education Committee. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Education activities shall be administered by the board of directors through an education committee.\n►<field>fi.lvl="3"</field>Section 5. General Manager. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Management of the business of the cooperative shall be vested in a general manager selected by and responsible to the board of directors. All regular employees subordinate to the general manager shall be engaged through nomination by the general manager and approval by the board of directors. The duties of the general manager shall be as follows:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To conduct the ordinary and usual business of the corporation, including the purchasing, marketing, and handling of all products and supplies handled by the corporation.\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To conduct such business in a manner that the members and patrons will receive just and fair treatment.\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To deposit all money belonging to the corporation that comes into the general manager\'s possession in the name of the corporation in a bank selected by the board of directors and, if authorized to do so by the board of directors, to make all disbursements by check from such account for the ordinary and necessary expenses of the business in the manner and form prescribed by the board of directors.\n►<field>fi.lvl="4"</field>d. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>To maintain all records and accounts in such a manner that the true and correct condition of the business may be ascertained at any time and to render annual and periodic statements in the form and manner prescribed by the board of directors.\n►<field>fi.lvl="4"</field>e. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Subject to the approval of the board of directors, to employ, supervise, and dismiss employees of the corporation but not agents or counsel specifically employed by the board of directors.\n►<field>fi.lvl="3"</field>Section 6. Annual Audit. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>An annual audit of accounts and a complete financial report shall be made at the annual membership meeting by an auditor selected by the board of directors.\n►<field>fi.lvl="3"</field>Section 7. Delegates to Other Organizations. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Delegates to [name of organization] or to any similar society of which this cooperative is a member shall be elected, each for a term of one year, by the board of directors.\n►<field>fi.lvl="2"</field>ARTICLE VI  MEMBERSHIP MEETINGS\n \n►<field>fi.lvl="3"</field>Section 1. Date of Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Regular membership meetings shall occur [specification of time interval] in [name of month 1] and [name of month 2].\n►<field>fi.lvl="3"</field>Section 2. Time and Place. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The time and place of each regular meeting shall be set by the board of directors. At least [number of days]-days prior to the date of any of these meetings, notice of the meeting, containing the time and place, shall be mailed to all members by the secretary or other person designated by the board of directors.\n►<field>fi.lvl="3"</field>Section 3. Order of Business. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The order of business at regular meetings shall consist of:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Roll call\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Proof of due notice of meeting\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Reading of minutes\n►<field>fi.lvl="4"</field>d. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>President\'s report\n►<field>fi.lvl="4"</field>e. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Manager\'s report\n►<field>fi.lvl="4"</field>f. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Treasurer\'s report\n►<field>fi.lvl="4"</field>g. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Auditor\'s report (at annual meetings only)\n►<field>fi.lvl="4"</field>h. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Committee reports\n►<field>fi.lvl="4"</field>i. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Elections as required in the bylaws\n►<field>fi.lvl="4"</field>j. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Unfinished business\n►<field>fi.lvl="4"</field>k. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>New business\n►<field>fi.lvl="4"</field>l. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Adjournment\n►<field>fi.lvl="3"</field>Section 4. Special Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Special meetings may be called by the board of directors. The meeting must be called by the secretary within [number of days] days after a written petition for such meeting, stating the time, place, and specific business to be discussed and signed by [percentage of voting members]&percnt;% of the voting members, has been presented to the secretary setting forth the meeting\'s purpose.\n►<field>fi.lvl="3"</field>Section 5. Notice of Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Written or printed notice of every regular or special meeting of the members will be prepared and mailed to the last-known post office address of each member not less than 10 days before the meeting. The notice will state the purposes of the meeting and the time and place that it is to be held. No business will be transacted at special meetings other than that referred to in the call.\n►<field>fi.lvl="3"</field>Section 6. Quorum. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>At any membership meeting, the presence of [number of persons having voting rights] persons having voting rights, or such lesser number of persons as shall equal [percentage of all members having voting rights]&percnt;% of all members having voting rights, shall constitute a quorum for the transaction of business. If balloting is conducted by mail on any issue or election, the number of members voting by mail shall be included in determining whether quorum requirements are satisfied with respect to matters included on the mail ballot.\n►<field>fi.lvl="3"</field>Section 7. Identification Card. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>At all business meetings, the secretary shall issue to each voting member, preferably at the time of entrance, an identification card, which shall be used in all show of hands or ballot voting.\n►<field>fi.lvl="3"</field>Section 8. Voting by Mail. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>In the event it shall be deemed advisable, the board of directors in the first instance, or when directed to do so at a membership meeting, may conduct a vote of the membership by mail on a specific issue or issues or an election. The procedure for conducting such balloting shall be established by the board of directors according to law.\n►<field>fi.lvl="2"</field>ARTICLE VII  DIRECTORS AND OFFICERS\n \n►<field>fi.lvl="3"</field>Section 1. Number of Directors; Election. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The board of directors shall consist of [number of members] members. Each director shall be a member of the corporation in good standing, and a person will not be eligible to become a director if he or she is in competition with or is affiliated with any enterprise that is in competition with the corporation. Each year at the [name of month] membership meeting, [number of directors] directors shall be elected from the nominees presented by the nominating committee or from the floor, each for a term of one year or until a successor is elected. Any member with voting rights is eligible for directorship except employees and members of their immediate families. At least two nominees will be nominated for each directorship. All directors will be elected by secret ballot and the nominee with the greatest number of votes will be elected.\n►<field>fi.lvl="3"</field>Section 2. Alternate Members. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Each year at the [name of month] meeting of the membership, there shall also be elected [number of alternate members] alternate members of the board. These members shall be the next [number of remaining members] remaining nominees for the board of directors having the highest vote after all vacancies on the board of directors have been filled. They shall have the privilege of participating in the discussion but have no power to vote. In case of an absence of a regular director at a given board meeting, the chairperson shall appoint the alternate present who received the highest number of votes to function as a director at the meeting. If a second regular director is absent from the given board meeting, the chairperson shall appoint the alternate present who received the second highest number of votes to function as a director at such meeting. Should a third regular director be absent from the given board meeting, the chairperson shall appoint the alternate who received the third highest number of votes to function as a director at such meeting. In case of a vacancy on the board, the board shall select one of the alternates to serve during the vacancy but not beyond the next [name of month] meeting.\n►<field>fi.lvl="3"</field>Section 3. Meetings; Quorum. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The directors shall have regular monthly meetings, and [number of directors] shall constitute a quorum for official business. A majority of the [total number of directors] directors is required for official action. All meetings shall be open to the members of the cooperative.\n►<field>fi.lvl="3"</field>Section 4. Absence from Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any director who is absent from [number of consecutive meetings] consecutive meetings of the board shall be dropped from the board at the next meeting unless excused by vote of the board, provided that the director shall have been notified of the situation by the secretary, such notice to be given after a director has missed [number of consecutive meetings] consecutive meetings.\n►<field>fi.lvl="3"</field>Section 5. Special Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>A special meeting of the board of directors will be held whenever called by the president or by a majority of the directors. Any business may be transacted at a special meeting. Each call for a special meeting will be in writing, signed by the person or persons calling for the meeting, addressed and delivered to the secretary, and will state the time and place of the meeting. On the signing of a waiver of notice of a meeting, a meeting of the board of directors may be held at any time.\n►<field>fi.lvl="3"</field>Section 6. Compensation. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The compensation, if any, of the members of the board of directors and of the executive committee will be determined by the members of the corporation at any regular or special meeting of the corporation. No member of the board of directors will occupy any position in the corporation on regular salary. The compensation, if any, of the corporation\'s officers will be determined by [specification of determination method].\n►<field>fi.lvl="3"</field>Section 7. Election of Officers. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>At the first meeting of the directors after each [name of month] membership meeting, the directors shall elect as officers of the cooperative [OPTIONAL: from among the directors] a president and a vice president. The directors shall also elect a treasurer and a secretary (or a secretary&ndash;-treasurer), who may, but need not, be members of the board or members of the cooperative.\n►<field>fi.lvl="3"</field>Section 8. Duties of Officers. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The duties of the officers shall be as follows:\n \n►<field>fi.lvl="4"</field>(a). Duties of the president. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The president shall preside over all meetings of the corporation and of the board of directors; call special meetings of the board of directors, as required; perform all acts and duties and render such reports as are usually required of an executive and presiding officer; sign certificates for shares or membership of the corporation and such other documents as the president may be authorized or directed to sign by the board of directors; and perform such other duties as the board of directors may prescribe.\n►<field>fi.lvl="4"</field>(b). Duties of the vice president. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>In the absence of disability of the president, the vice president shall perform the duties of the president.\n►<field>fi.lvl="4"</field>(c). Duties of the secretary. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The secretary shall keep a record of all meetings of the corporation and of the board of directors; keep complete membership records and have general charge of the corporation\'s books and records; sign all membership or stock certificates with the president and sign such other papers as the secretary may be authorized or directed to sign by the board of directors; serve all notices required by law and these bylaws and make all reports required by law; keep the corporate seal and all books of blank certificates complete; and affix the corporate seal to all papers requiring such seal.\n►<field>fi.lvl="4"</field>(d). Duties of the treasurer. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The treasurer will perform such duties with respect to the finances of the corporation as may be prescribed by the board of directors.\n►<field>fi.lvl="3"</field>Section 9. Powers of Directors. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The board of directors shall have full power in administrating the cooperative\'s affairs and may make such rules and regulations with respect to the business and employees of the cooperative as the board deems advisable, provided that all actions shall be consistent with these bylaws.\n►<field>fi.lvl="3"</field>Section 10. Committees. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The president shall appoint the following standing committees of the board immediately after taking office: [names of standing committees], as well as such other special committees as may be deemed necessary by the board of directors.\n►<field>fi.lvl="3"</field>Section 11. Loans Involving Mortgages. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The board shall not accept any loan involving a mortgage on any of the property of the cooperative, except as authorized at a membership meeting with previous notice of the proposed loan.\n►<field>fi.lvl="3"</field>Section 12. Other Loans. \n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The cooperative, by action of the board of directors, may accept other loans, provided such loans do not in the aggregate exceed the cooperative\'s net worth, except as authorized at a membership meeting with previous notice of the proposed loan or by mail ballot.\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Loans from members and others shall be solicited only when sufficient capital cannot be obtained from membership shares.\n►<field>fi.lvl="3"</field>Section 13. Bonds. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Fidelity bonds shall be required of the treasurer and manager.\n►<field>fi.lvl="3"</field>Section 14. Suggested Order of Business. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Suggested order of business at regular meetings of the directors is:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Minutes of the preceding meeting\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Unfinished business\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Communications and bills\n►<field>fi.lvl="4"</field>d. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Reports of officers\n►<field>fi.lvl="4"</field>e. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Report of manager\n►<field>fi.lvl="4"</field>f. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Reports of committees\n►<field>fi.lvl="4"</field>g. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>New business\n►<field>fi.lvl="2"</field>ARTICLE VIII  NOMINATIONS AND ELECTIONS\n \n►<field>fi.lvl="3"</field>Section 1. Limitation on Consecutive Terms. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>No member shall be elected as a director for more than [number of consecutive full terms] consecutive full terms.\n►<field>fi.lvl="3"</field>Section 2. Nominating Committee. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>A nominating committee of [number of members] members shall be elected annually at the [name of month] membership meeting.\n►<field>fi.lvl="3"</field>Section 3. Nominations by Committee. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The nominating committee shall make nominations for all positions, whether as regular or alternate directors, to be filled by election of the members. It shall submit a list of such nominations, the number of which shall be at least [percentage of total number of positions to be filled]&percnt;% greater than the total number of positions to be filled, to the secretary of the cooperative by the [day of week] following the [ordinal number of day] [day of week] of [name of month]. The committee may not nominate any of its own members. Such members may, however, be nominated by petition or from the floor, as provided by Section 4 of this ARTICLE.\n►<field>fi.lvl="3"</field>Section 4. Nomination by Members. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Nominations may also be made by petition of [number of members] or more members submitted to the secretary of the cooperative by the [ordinal number of day] of [name of month] in any year. Such nominees will be provided equal opportunity with other nominees to present their views. Each voting member of record shall be notified of all nominations not less than [number of days] days before an election is held and shall be informed which nominations were made by the committee and which by petition. Additional nominations may be made from the floor.\n►<field>fi.lvl="2"</field>ARTICLE IX  MANAGERS AND EMPLOYEES\n \n►<field>fi.lvl="3"</field>Section 1. Manager\'s Authority. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The manager shall be in full charge of the business of the cooperative but shall conduct the business in conformity with policies determined by the board of directors and the membership and in accordance with regulations and instructions of the board of directors as the administrative agency of the cooperative.\n►<field>fi.lvl="3"</field>Section 2. Reports to Board; Financial Statements. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The manager shall attend all meetings of the board of directors and shall make such reports as are desired by the board. The manager shall prepare and submit a monthly financial statement to the board.\n►<field>fi.lvl="3"</field>Section 3. Report to Membership; Inventories. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The manager shall attend membership meetings and make such reports as are requested by the president or by action taken in membership meetings. In [name of month of semi-annual inventory], the manager shall take a complete inventory. At the [name of month] meeting of the directors and at the [name of month] membership meeting, the manager shall present a statement of operations and a balance sheet. At the [name of month] meeting of the directors and the [name of month] membership meeting, the manager shall make corresponding reports for the year. In each case, the manager shall have available copies for inspection by anyone interested, and a permanent copy shall be filed in the cooperative\'s registered office.\n►<field>fi.lvl="3"</field>Section 4. Employee Grievances. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any employee or representative of the employees shall have the right to present any grievances directly to the board of directors.\n►<field>fi.lvl="3"</field>Section 5. Suspension of Employees. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The manager must give an employee [number of days] days\' notice of intention of terminating that employee\'s services. For serious cause, such as dishonesty or other immoral conduct, the board of directors may immediately suspend an employee. In either case, the employee shall have the right of a hearing before the board and the right of appeal from the decision of the board as provided in Section 4 of ARTICLE X on initiative and referendum. This section shall be subject to any agreement that this cooperative shall have entered into with any labor union.\n►<field>fi.lvl="2"</field>ARTICLE X  PATRONS AND PATRONS\' NET MARGINS\n \n►<field>fi.lvl="3"</field>Section 1. Patrons. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Each patronage transaction between this corporation and each patron will be subject to and will include as a part of its terms each provision of this ARTICLE X, whether it be expressly referred to in the transaction or not, and no person shall have any authority to transact any business for this corporation with any patron on any terms inconsistent with this ARTICLE X. On selling, consigning, or otherwise delivering any agricultural product to this corporation or contracting to do so and on buying or otherwise receiving any farm supplies or equipment from this corporation or contracting to do so, each patron, with or without then executing any writing or doing any other act, thereby:\n \n►<field>fi.lvl="4"</field>(a). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Shall, as further consideration due the patron from this corporation on account of such transaction, become entitled to have paid such proportion of the patrons\' net margins received by this corporation as such patronage bears to the aggregate patronage of all patrons, as is in this document more particularly defined and provided; and\n►<field>fi.lvl="4"</field>(b). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Shall, in consideration of similar agreements by others in the same fiscal year, promise and agree to invest in the capital of this corporation (revolving fund) as requested by the board of directors [specification of determinant] of the agricultural products sold, consigned, or otherwise delivered to the corporation, which amount is made deductible from the sales proceeds otherwise payable to the patron, and, in addition, such further sum or sums of money as the board of directors may specify, provided, however, that the patron\'s obligation to invest in the capital in any fiscal year of this corporation, in addition to the rate prescribed above, shall be limited to an amount equal to the patron\'s share of such patrons\' net margins for the fiscal year next preceding the fiscal year in which such investment is required and shall be proportionate to the investments required of other patrons. It is the intention and agreement of this corporation and its patrons with each other that, as the board of directors shall deem the capital to be necessary to repurchase or retire previously issued certificates of indebtedness or capital interests, its several patrons shall, subject to the foregoing limitations, furnish such additional capital as may be required and shall do so substantially in proportion to their respective patronage in the preceding fiscal year.\n►<field>fi.lvl="3"</field>Section 2. Computation of Patrons\' Net Margins. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The patrons\' net margins, calculated on the basis of each fiscal year, shall be computed as follows:\n \n►<field>fi.lvl="4"</field>(a). Gross receipts. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>All proceeds of all sales of products marketed for patrons, plus all sums received for supplies and equipment procured for patrons, plus all sums received from all other sources except loans or contributions to this corporation or investments in its capital, will be deemed to be the &ldquo;“gross receipts.&rdquo;”\n►<field>fi.lvl="4"</field>(b). Patrons\' net margins. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>This corporation will deduct from the gross receipts the sum of the following items:\n \n►<field>fi.lvl="5"</field>(1). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>All costs and expenses and other charges that are lawfully excludible or deductible from this corporation\'s gross receipts for the purpose of determining the amount of any net margin of this corporation; and\n►<field>fi.lvl="5"</field>(2). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>An amount equal to [percentage of balance of gross receipts]&percnt;% of the balance of the gross receipts that remain after first having deducted from it the items referred to in Clause (1).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>If, in any fiscal year, this corporation incurs a net operating loss that is recognizable for tax purposes, the board of directors will have full authority to charge off such loss against net margins of future years or against revolving fund credits or in such other manner as will afford the corporation the maximum benefit for tax purposes.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The balance of the gross receipts that remain after the foregoing deductions will be deemed to be the &ldquo;“patrons\' net margins.&rdquo;”\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The amount deducted pursuant to Clause (2) above will be set aside in a reserve and apportioned to patrons in accordance with their patronage during the fiscal year, provided that, when the reserve reaches the sum of $[dollar amount of reserves], the board of directors in its discretion may either: (a) make no further deductions; or (b) retire such amounts of the reserve as it considers may be in excess of the corporation\'s reserve needs. Such retirement will be accomplished by paying off in full or pro rata patrons\' apportioned interests, by years, the oldest being paid first.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The patrons\' respective shares of the patrons\' net margins will be computed on the basis of their respective patronage of, and the net margins resulting from the operations of, the various pools or departments of this corporation and will be in proportion to the quantity or value of the products delivered by, or supplies or equipment procured for, such patrons.\n►<field>fi.lvl="3"</field>Section 3. Payment of Patrons\' Net Margins. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Each patron\'s share of the patrons\' net margins will be payable to the patron in cash at the close of each fiscal year, but the legal offset of the payments due a patron against the obligation of such patron to subscribe for capital, as provided in Section 1(b) of this ARTICLE X, will constitute full and complete discharge of the obligation of the corporation to make payment as provided in these bylaws.\n►<field>fi.lvl="3"</field>Section 4. Lien. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>This corporation will have a first lien on the interest of each patron in the patrons\' net margins for all indebtedness of such patron to this corporation.\n►<field>fi.lvl="2"</field>ARTICLE XI  REVOLVING FUND\n \n►<field>fi.lvl="3"</field>Section 1. Purpose. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>This corporation will establish and maintain a revolving fund for the purpose of acquiring and maintaining adequate capital to finance its business.\n►<field>fi.lvl="3"</field>Section 2. Investment in Fund. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>This corporation may require investment in its revolving fund as provided in ARTICLE X, Section 1(b) of these bylaws. Proper entries will be made on the corporation\'s books and records so that the new credit to each holder of an interest in the revolving fund in each year can be ascertained at any time. This corporation may issue revolving fund certificates to evidence credits in such fund, and the certificates may be in such form and contain such terms and conditions not inconsistent with this ARTICLE XI as the board of directors may prescribe.\n►<field>fi.lvl="3"</field>Section 3. Operation. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Investments in the revolving fund need not be segregated from, but on the contrary may be invested in or commingled with, any other assets of this corporation. No dividend, interest, or any other income will be declared or paid on account of any net credits in the revolving fund. This corporation will have a first lien on each revolving fund credit for all indebtedness of the holder thereof to this corporation.\n►<field>fi.lvl="3"</field>Section 4. Repayment. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>When, in the judgment of the board of directors, the net balance in the revolving fund exceeds the amount of capital reasonably needed by the corporation, it will apply such excess to the retirement, in full or pro rata, of all the net credits in the revolving fund, provided that such certificates will be retired in the order of issuance by years and there will be no discrimination between investments received in the same fiscal year. Notwithstanding any other provision of this section, the board of directors, in its discretion, may retire any net credits at any time when the holder: (a) dies; (b) otherwise ceases to be a producer; or (c) moves from the territory served by the corporation.\n►<field>fi.lvl="3"</field>Section 5. Transfer. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any assignment or transfer of any revolving fund will not be binding on this corporation without the consent of the board of directors or until it shall have been entered in the books of this corporation.\n►<field>fi.lvl="2"</field>ARTICLE XII  INITIATIVE, REFERENDUM, AND RECALL\n \n►<field>fi.lvl="3"</field>Section 1. Regulations for Mail Ballots. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Regulations for conducting mail ballots under this section shall be made by the board of directors.\n►<field>fi.lvl="3"</field>Section 2. Petition for Director\'s Recall; Balloting. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any director may be recalled by the following method: On petition of at least [percentage of voting members]&percnt;% of the voting members, accompanied by written charges, the secretary or other officer specified in the petition shall conduct a recall election at a regular or special meeting. The ballot shall contain two statements of not more than [number of words] words each, one by a proponent and one by an opponent of the recall. The director against whom the charges have been brought must be given at least 20-days written notice of the time, place, and object of any such meeting and of the charges to be brought, and the director shall be given an opportunity to be heard at the meeting and to present and cross-examine witnesses. A three-fourths vote of at least [percentage of voting members]&percnt;% of the entire membership shall determine the election.\n►<field>fi.lvl="3"</field>Section 3. Referendum on Certain Motions. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>On demand of one-third of the entire board of directors made immediately and so recorded at the same meeting at which the original motion was passed, any matter that has been approved or passed on by the board must be referred to the entire membership for a decision at the next regular or special meeting.\n►<field>fi.lvl="2"</field>ARTICLE XIII  AMENDMENTS\n \n►<field>fi.lvl="3"</field>Section 1. Proposal. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Amendments to these bylaws may be proposed:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>By the board of directors, acting on its own motion or on the petition of any member, provided that if such a petition is signed by at least [number of members] members and submitted to the board of directors at a board meeting not less than [number of days] days before a membership meeting, the board of directors must submit the proposed amendment to the members by mail, as provided below, and the proposed amendment must be voted on at the next membership meeting; or\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>By any member submitting the proposed amendment in writing supported by a petition of at least [number of members] members at any regular membership meeting. The board of directors must notify the membership of the proposed amendment at least [number of days]-days prior to the next membership meeting, and the proposed amendment must be voted on at the next membership meeting.\n►<field>fi.lvl="3"</field>Section 2. Adoption. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Amendment to these bylaws may be adopted by the written consent of members entitled to exercise a majority of the voting power or by the vote of a majority of a quorum at a meeting of members duly called. Such meeting shall be considered duly called if the proposed amendment has been submitted in writing at the next preceding regular membership meeting or has been submitted to the members by mail at least [number of days]-days prior to the membership meeting at which a vote on the amendment is taken.\n►<field>fi.lvl="2"</field>ARTICLE XIV  DISSOLUTION\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>This cooperative may be dissolved only by a [fraction of votes] vote of all voting members of the entire membership at any regular or special meeting when due notice of such action has been given in advance. On dissolution, any reserves of the cooperative in excess of its financial obligation, including debts and expenses, par value of invested capital paid on subscriptions, patronage dividends and membership capital, shall be turned over to such recognized cooperative organization as the membership may determine.\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of bylaws]\n ►<field>fsig.pos="r"</field>►<field>fal.lo="n"</field>[Name of corporation]\n ►<field>fal.lo="n"</field>By:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of authorized representative]\n ►<field>fal.lo="n"</field>[Title of authorized representative]\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Attested:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of secretary]\n ►<field>fal.lo="n"</field>Secretary\n ►<field>fal.lo="n"</field>[Corporate seal]\nNotes to Form\n\nTax Notes \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Sound tax planning for specialized entities such as a cooperative usually advocates inclusion in the articles of incorporation or bylaws of the goal of complying with the tax requirements. That demonstrates that the purpose of the entity is to meet the tax requirements and makes easier its claim to the specialized tax treatment afforded by the Internal Revenue Code. As an example, the key to allowance of the patronage dividend exclusion is a pre-existing enforceable obligation of the co-op to distribute its patronage earnings among its patrons. ►<field>st.ref.id="I049425"</field>26 U.S.C.A. &s;§1388(a)(2). Listing this in the bylaws not only shows that the corporation plans to do this but also makes it a pre-existing obligation of the corporation. If this should not be covered elsewhere, it nevertheless provides compliance with the Code.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Further examples of the policy of covering tax requirements in the bylaws may be found in the foregoing bylaws. See, for example, Sec 4(h) of ARTICLE III and Secs 1(c), (e), and (h) of ARTICLE IV.\nResearch References\n ►<field>tk.ref.id="I04BX71"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04EQBX"</field>West\'s Key Number Digest, Associations &key;128, 129\nARTICLE I. \nSection 1. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The name shall include the word &ldquo;“Cooperative.&rdquo;” ►<field>st.ref.id="I04HJGT"</field>Coop. Corp. Law &s;§11(1).\nResearch References\n ►<field>rc.ref.id="I05QHBL"</field>N.Y. Jur. 2d, Business Relationships &ss;§§164 to 179\nARTICLE III. \nSection 1. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Only persons engaged in the production of agricultural products, or cooperative corporations of such producers organized under the laws of this or any other state, shall be eligible for membership in any agricultural marketing or purchasing corporation formed or operated under the provisions of this Article. The terms and conditions of membership shall be prescribed in the bylaws. See ►<field>st.ref.id="I05QHBI"</field>Coop. Corp. Law &s;§111.\nResearch References\n ►<field>rc.ref.id="I05NO6M"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\n ►<field>rc.ref.id="I05KV1Q"</field>N.Y. Jur. 2d, Business Relationships &s;§825\nARTICLE III. \nSection 3(b)(1). \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>As to tax deductions for cooperatives, see ►<field>st.ref.id="I05I1WU"</field>26 U.S.C.A. &s;§1382(b), (c).\nARTICLE III. \nSection 7. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>In accordance with provisions in the bylaws, any person shall: (1) forfeit his or her membership upon proof that he or she has ceased to be engaged in the occupation or occupations for the servicing of which the corporation was formed or has ceased to have the qualifications requisite for membership; and (2) upon such notice and terms as may be prescribed in the bylaws, surrender his or her membership certificate or his or her shares of stock upon payment of the par or otherwise designated value thereof and of any accrued dividends thereon, as may appear in the accounting at the end of the current fiscal year. ►<field>st.ref.id="I05QHBG"</field>Coop. Corp. Law &s;§42.\nResearch References\n ►<field>rc.ref.id="I05NO6K"</field>N.Y. Jur. 2d, Business Relationships &s;§836\nARTICLE III. \nSection 8. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any member of a nonstock corporation may, subject to fulfilling the liability, contractual or otherwise, then incurred by him or her as a member of such corporation, resign and withdraw from such corporation, in accordance with provisions therefor in the bylaws. ►<field>st.ref.id="I05KV1O"</field>Coop. Corp. Law &s;§43.\nResearch References\n ►<field>rc.ref.id="I05I1WS"</field>N.Y. Jur. 2d, Business Relationships &s;§836\nARTICLE IV. \nSection 1(c). \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Return on membership capital may not exceed 12&percnt;% per year. ►<field>st.ref.id="I05F8RW"</field>Coop. Corp. Law &s;§111(a).\nARTICLE V. \nSection 7. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any corporation may, on resolution adopted by its board of directors, enter into all necessary and proper contracts and agreements with any other cooperative corporation or association. ►<field>st.ref.id="I05QHAB"</field>Coop. Corp. Law &s;§14(l).\nResearch References\n ►<field>rc.ref.id="I05NO5F"</field>N.Y. Jur. 2d, Business Relationships &ss;§§472 to 476\nARTICLE VII. \nSection 1. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The bylaws may provide for the qualifications, compensation, duties, and term of office of directors; the time of their election; and the mode and manner of giving notice. Directors shall be elected at the first annual meeting in three classes; the first class shall serve for a period of one year, the second class shall serve for two years, and the third class shall serve for three years. Thereafter, all successors will be elected for a period of three years. ►<field>st.ref.id="I05QHA8"</field>Coop. Corp. Law &s;§60.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The first annual meeting must be held within a reasonable time so as to give the members or the shareholders an opportunity to elect their own board as promptly as possible, and a member or shareholder is entitled to an order compelling such meeting if necessary. ►<field>cc.ref.id="I05NO5C"</field>Lazar v. Knolls Co-op. Section No. 2, 205 Misc. 748, 130 N.Y.S.2d 407 (Sup 1954).\nARTICLE VII. \nSection 7. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The officers of every cooperative corporation must include a president, one or more vice presidents, secretary, and treasurer who are required to be elected annually by the board of directors. The president and a first vice president must be members of the corporation or of a member corporation and must be elected from among the directors. Other officers need not be directors or members of the corporation. The offices of secretary and treasurer may be combined. ►<field>st.ref.id="I05KV0G"</field>Coop. Corp. Law &s;§64.\nResearch References\n ►<field>rc.ref.id="I05I1VK"</field>N.Y. Jur. 2d, Business Relationships &ss;§§532, 536\nARTICLE VII. \nSection 13. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Before handling funds or securities of the corporation, amounting to $1,000 or more in any one year, the officer or employee shall be covered by an adequate bond to be approved by the board of directors. ►<field>st.ref.id="I05QHA6"</field>Coop. Corp. Law &s;§65.\nResearch References\n ►<field>rc.ref.id="I05NO5A"</field>N.Y. Jur. 2d, Business Relationships &s;§687\nARTICLE X. \nSection 3. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>A cooperative that does not make full cash payment, utilizing instead qualified written notices of allocation (making at least 20&percnt;% cash payment of the patronage dividend), must obtain the written consent of the recipient to take into account the stated dollar amount of the patronage distribution for federal income tax purposes. One method of obtaining such consent is through the cooperative adopting an appropriate bylaw and otherwise following the procedure laid down in the federal statutes. See ►<field>st.ref.id="I05KV0E"</field>26 U.S.C.A. &ss;§§1381, 1382, 1385, 1388.\nResearch References\n Qualified written notice of allocation, ►<field>rc.ref.id="I04949A"</field>Federal Tax Coordinator 2d &p;E-1121\n Noncash patronage dividends, ►<field>rc.ref.id="I04BXE6"</field>Federal Tax Guide to Legal Forms &s;§5:18\nARTICLE XII. \nSection 2. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Any member may bring charges against a director by filing them in writing with the secretary, together with a petition signed by 5&percnt;% of the members, requesting removal. The corporation may thereupon remove the director by the affirmative vote of three-fourths of the members voting thereon at a meeting promptly held after due notice in writing, setting forth accurately the purpose for which such meeting is called, provided that at such meeting not less than 10&percnt;% of the entire membership vote, personally or by mail. The director involved shall be given a copy of the charges reasonably in advance of the meeting, and he or she and the complainant shall have an opportunity at the meeting to be heard in person or by counsel and to present and cross-examine witnesses. ►<field>st.ref.id="I04949C"</field>Coop. Corp. Law &s;§63.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>In case the bylaws provide for election of directors by districts with primary elections in each district, then the petition for removal of a director must be signed by 20&percnt;% of the members residing in the district from which such director was elected. The board of directors must call a special meeting of the members residing in that district to consider the removal of the director, and, by a concurrent vote of a majority of the members of the district voting on the issue, the director will be removed from office. ►<field>st.ref.id="I04BXE8"</field>Coop. Corp. Law &s;§63.\nResearch References\n ►<field>rc.ref.id="I04EQJ4"</field>N.Y. Jur. 2d, Business Relationships &s;§569\nARTICLE XIII. \nSection 1. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>The bylaws must provide a method for the bylaws to be amended, see ►<field>st.ref.id="I04HJO0"</field>Coop. Corp. Law &s;§16.\nResearch References\n ►<field>rc.ref.id="I04KCSW"</field>Construction and effect of corporate bylaws or articles relating to change in number of directors, 3 A.L.R.3d 623\n ►<field>rc.ref.id="I04949F"</field>N.Y. Jur. 2d, Business Relationships &s;§128\nARTICLE XIV. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>A cooperative corporation may, at any meeting and upon due and express notice previously given, by vote of two-thirds of all of the members or stockholders voting thereon, discontinue its operations and settle its affairs. ►<field>st.ref.id="I04BXEB"</field>Coop. Corp. Law &s;§17.\nResearch References\n ►<field>rc.ref.id="I04EQJ7"</field>N.Y. Jur. 2d, Business Relationships &ss;§§1337, 1432\nARTICLE II. Section 1. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|23|32:23"</field>Under ARTICLE II., Section 1, description of business may include name of agricultural product.\n32:24. Bylaws of agricultural cooperative marketing corporation-- Stock\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39758c0181a11e68738d3db3a7c8dd3"</field>\n►<field>fi.lvl="1"</field>BYLAWS OF [NAME OF AGRICULTURAL COOPERATIVE ASSOCIATION], A NEW YORK AGRICULTURAL COOPERATIVE ASSOCIATION\n \n►<field>fi.lvl="2"</field>ARTICLE I  MEMBERS AND MEMBERSHIP\n \n►<field>fi.lvl="3"</field>Section 1. Eligibility. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Persons engaged in the production of agricultural products and cooperative corporations may become shareholders of this corporation by purchasing at least one fully paid membership share and by agreeing to comply with the bylaws.\n►<field>fi.lvl="3"</field>Section 2. Termination of Membership. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Any shareholder may be expelled by the vote of a majority of the shareholders voting at a regular or special meeting, provided:\n \n►<field>fi.lvl="4"</field>a. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The notice of the meeting specifically states that the expulsion of the shareholder will be considered;\n►<field>fi.lvl="4"</field>b. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The shareholder whose expulsion is to be considered is notified in writing of that fact at least [number of days] days in advance of the meeting by a notice sent by registered mail, postage prepaid, to the shareholder\'s last-known address as it appears on the corporation\'s records; and\n►<field>fi.lvl="4"</field>c. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The shareholder is given an opportunity to be heard either in person or by counsel at the meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Expulsion may be for such cause as a majority of the shareholders voting at the meeting may deem sufficient. On expulsion of a shareholder, the corporation shall repurchase the membership shares held by the shareholder if and when, in the board of directors\' sole discretion, there are sufficient funds available.\n►<field>fi.lvl="2"</field>ARTICLE II  MEETINGS\n \n►<field>fi.lvl="3"</field>Section 1. Regular. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>A regular meeting of the shareholders shall be held between the [ordinal number of day 1] day of [name of month 1] and the [ordinal number of day 2] day of [name of month 2] of each year.\n►<field>fi.lvl="3"</field>Section 2. Special. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Special meetings of the shareholders may be called by the board of directors, and a special meeting must be called by the secretary in response to a petition signed by [percentage of shareholders]&percnt;% of the shareholders.\n►<field>fi.lvl="3"</field>Section 3. Notice. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Notice of all special meetings of shareholders shall be given at least [number of days] days before the meeting by mailing a notice to each shareholder at the shareholder\'s last-known address.\n►<field>fi.lvl="3"</field>Section 4. Quorum. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>At any meetings of the shareholders, the presence in person of [number of shareholders having voting rights] shareholders having voting rights or such lesser number of persons as shall equal [percentage of all shareholders having voting rights]&percnt;% of all shareholders having voting rights constitutes a quorum for the transaction of business. If the written votes of absent shareholders are received and counted, the number of absent shareholders so voting shall be included in determining whether quorum requirements are satisfied with respect to matters upon which such votes are cast.\n►<field>fi.lvl="3"</field>Section 5. Voting Rights. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Each shareholder of the corporation is entitled to cast one vote and no more, regardless of the number of membership shares held by the shareholder. In the case of membership shares held in joint ownership by two or more persons, any one of such persons may cast the vote. Members may vote in person or by mail, subject to rules and regulations set forth by the board of directors.\n►<field>fi.lvl="2"</field>ARTICLE III  DIRECTORS\n \n►<field>fi.lvl="3"</field>Section 1. Conduct of Affairs. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Except as otherwise provided by the articles of incorporation or these bylaws, the powers of this corporation shall be exercised, its property controlled, and its affairs conducted by a board of [number of directors] directors.\n►<field>fi.lvl="3"</field>Section 2. Election. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The persons named as directors shall serve until the first annual meeting of the shareholders. At that meeting, the shareholders shall nominate and elect [number of directors] directors. In all cases, directors shall serve until their successors are elected.\n►<field>fi.lvl="3"</field>Section 3. Vacancies. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Any vacancy occurring in the board of directors, except by recall, shall be filled by appointment by a majority of the remaining board shareholders though they constitute less than a quorum. Any director so appointed shall hold office only until [his/her] successor is elected at the next annual meeting of the shareholders.\n►<field>fi.lvl="3"</field>Section 4. Regular Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The board of directors shall hold regular meetings at least once a month at a time and place previously determined by the board.\n►<field>fi.lvl="3"</field>Section 5. Special Meetings. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Special meetings may be called at any time by the board of directors. [Text percentage of members of shareholders] percent of the members of shareholders may file a petition stating the specific business to be brought before the corporation and demand a special meeting at any time. Such meeting must then be called by the directors. Notice of all special meetings, together with a statement of the purpose or purposes, must be mailed to each shareholder at least [number of days]-days prior to the meeting.\n►<field>fi.lvl="3"</field>Section 6. Quorum. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>At any meeting of the board of directors, [a majority of/[fraction of directors]] directors shall constitute a quorum for the transaction of business.\n►<field>fi.lvl="3"</field>Section 7. Compensation. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Directors and officers shall receive no compensation for performing the duties of their respective offices, except as authorized by the shareholders.\n►<field>fi.lvl="3"</field>Section 8. Removal of Directors. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Any shareholder may bring charges against a director by filing them in writing with the secretary of the corporation, together with a petition signed by [percentage of shareholders]&percnt;% of the shareholders, requesting the removal of the director in question. The removal shall be voted on at the next regular or special meeting of the corporation, and by a vote of three-fourths of the shareholders in attendance, provided that not less than [percentage of entire membership vote]&percnt;% of the entire membership vote personally or by mail, the corporation may remove the director and fill the vacancy. The director against whom such charges have been brought will be informed in writing of the charges prior to the meeting and will have an opportunity at the meeting to be heard in person or by counsel and to present and cross examine witnesses, and the person bringing the charges against such director shall have the same opportunity.\n►<field>fi.lvl="2"</field>ARTICLE IV  OFFICERS\n \n►<field>fi.lvl="3"</field>Section 1. Officers. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The officers shall be a president and a vice president, who shall be directors, and a secretary and a treasurer who need not be directors or shareholders of the cooperative. Their duties shall be as follows:\n \n►<field>fi.lvl="4"</field>(a). Duties of the president. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The president shall preside over all meetings of the corporation and of the board of directors; call special meetings of the board of directors, as required; perform all acts and duties, and render such reports as are usually required of an executive and presiding officer; sign certificates for shares of the corporation and such other documents as the president may be authorized or directed to sign by the board of directors; and perform such other duties as the board of directors may prescribe.\n►<field>fi.lvl="4"</field>(b). Duties of the vice president. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>In the absence or disability of the president, the vice president shall perform the duties of the president.\n►<field>fi.lvl="4"</field>(c). Duties of the secretary. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The secretary shall keep a record of all meetings of the corporation and of the board of directors; keep complete membership records and have general charge of the corporation\'s books and records; sign all stock certificates with the president and sign such other papers as the secretary may be authorized or directed to sign by the board of directors; serve all notices required by law and these bylaws and make all reports required by law; and keep the corporate seal and all books of blank certificates complete and affix the corporate seal to all papers requiring such seal.\n►<field>fi.lvl="4"</field>(d). Duties of the treasurer. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The treasurer will perform such duties with respect to the finances of the corporation as may be prescribed by the board of directors.\n►<field>fi.lvl="3"</field>Section 2. Election. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Officers shall be elected annually by the board of directors at the first meeting of the shareholders.\n►<field>fi.lvl="3"</field>Section 3. Removal. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Any shareholder may bring charges against an officer by filing them in writing with the secretary of the corporation, together with a petition signed by [percentage of shareholders]&percnt;% of the shareholders, requesting the removal of the officer in question. The removal shall be voted on at the next regular or special meeting of the corporation, and by a vote of a majority of the shareholders, the corporation may remove the officer and fill the vacancy. The officer against whom such charges have been brought will be informed in writing of the charges prior to the meeting and will have an opportunity at the meeting to be heard in person or by counsel and to present witnesses, and the person bringing the charges against such director shall have the same opportunity.\n►<field>fi.lvl="3"</field>Section 4. Vacancies. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Any office that becomes vacant will be filled by election of the board of directors.\n►<field>fi.lvl="2"</field>ARTICLE V  ACCOUNTING\n \n►<field>fi.lvl="3"</field>Section 1. Fiscal Year. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The books of the corporation shall be kept on the basis of a fiscal year ending on [end date of fiscal year].\n►<field>fi.lvl="3"</field>Section 2. Annual Audit. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The board of directors shall arrange for an annual audit of the books and records of the corporation by a certified public accountant.\n►<field>fi.lvl="2"</field>ARTICLE VI  MEMBERSHIP SHARES\n \n►<field>fi.lvl="3"</field>Section 1. Issuance. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>No share shall be issued until its par value has been paid in full. All shares shall be consecutively numbered in the name and address of each shareholder, together with the number of shares owned and the dates of issue, and shall be entered on the corporation\'s books.\n►<field>fi.lvl="3"</field>Section 2. Joint Ownership. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Membership shares may only be jointly owned by two or more persons.\n►<field>fi.lvl="3"</field>Section 3. Transfer of Shares. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Membership shares shall be nontransferable except to persons engaged in the production of agricultural products, and such restrictions shall be printed on every certificate of common stock.\n►<field>fi.lvl="3"</field>Section 4. Purchase by Corporation. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The corporation has the right to cancel the membership and to purchase, at par value, the shares of any shareholder who has failed to patronize the corporation as a producer or consumer for a period of [number of days] days after giving [number of days]-days\' notice by registered mail to the shareholder at the shareholder\'s last-known address. The corporation may also, in the discretion of the board of directors, purchase, at par value, all or part of the shares of a shareholder who makes a written request for such purchase. Sale to the corporation by any shareholder of all of such shareholder\'s shares will constitute resignation from membership.\n►<field>fi.lvl="2"</field>ARTICLE VII  DISTRIBUTION OF NET INCOME\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Net income in excess of additions to reserves and surpluses shall be distributed to the shareholders of the cooperative on the basis of patronage by action of the board of directors. However, in accordance with state law governing cooperative corporations, dividends on membership stock, if earned and declared, shall not exceed [percentage of dividends]&percnt;% per year.\n►<field>fi.lvl="2"</field>ARTICLE VIII  REFERENDUM\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>On demand of one-third of the entire board, made immediately and recorded at the same meeting at which the original motion was passed, any matter passed by the board must be referred to all shareholders for decision at the next regular or special meeting.\n►<field>fi.lvl="2"</field>ARTICLE IX  AMENDMENTS TO BYLAWS\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>A majority vote of a quorum of shareholders attending a meeting, of which notice of the proposed bylaws has been given, is sufficient to adopt or amend the bylaws.\nNotes to Form\n\nTax Notes \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Sound tax planning for specialized entities such as a cooperative usually advocates inclusion in the articles of incorporation or bylaws of the goal of complying with the tax requirements. That demonstrates that the purpose of the entity is to meet the tax requirements and makes easier its claim to the specialized tax treatment afforded by the Internal Revenue Code. As an example, the key to allowance of the patronage dividend exclusion is a pre-existing enforceable obligation of the co-op to distribute its patronage earnings among its patrons. ►<field>st.ref.id="I09SME0"</field>26 U.S.C.A. &s;§1388(a)(2). Listing this in the bylaws not only shows that the corporation plans to do this but also makes it a pre-existing obligation of the corporation. If this should not be covered elsewhere, it nevertheless provides compliance with the Code.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Cooperative corporations may issue either stock (as in these bylaws) or membership shares. For a form of bylaws containing membership provisions, see ►<field>x.ref.id="I04BXLD"</field>&s;§32:23.\nResearch References\n ►<field>tk.ref.id="I04EQQ9"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04HJV5"</field>West\'s Key Number Digest, Associations &key;128, 129\nARTICLE I. \nSection 1. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Only persons engaged in the production of agricultural products, or cooperative corporations of such producers organized under the laws of this or any other state, shall be eligible for membership in any agricultural marketing or purchasing corporation formed or operated under the provisions of this Article. The terms and conditions of membership shall be prescribed in the bylaws. See ►<field>st.ref.id="I04KD01"</field>Coop. Corp. Law &s;§111(a).\nResearch References\n ►<field>rc.ref.id="I0494GK"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\n ►<field>rc.ref.id="I04BXLG"</field>N.Y. Jur. 2d, Business Relationships &s;§825\nARTICLE I. \nSection 2. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>In accordance with provisions in the bylaws, any person: (1) shall forfeit his or her membership upon proof that he or she has ceased to be engaged in the occupation or occupations for the servicing of which the corporation was formed or has ceased to have the qualifications requisite for membership; and (2) shall, upon such notice and terms as may be prescribed in the bylaws, surrender his or her membership certificate or his or her shares of stock upon payment of the par or otherwise designated value thereof and of any accrued dividends thereon, as may appear in the accounting at the end of the current fiscal year. ►<field>st.ref.id="I04EQQC"</field>Coop. Corp. Law &s;§42.\nResearch References\n ►<field>rc.ref.id="I04HJV8"</field>N.Y. Jur. 2d, Business Relationships &s;§836\nARTICLE III. \nSection 7. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>A cooperative corporation may provide a fair remuneration for its directors and for members of its executive committee. No director or member of the executive committee shall be a party to a contract for profit with the corporation that, in substance, shall differ in any way from similar contracts by it with members or with others or that shall vary from terms generally current in the district. ►<field>st.ref.id="I05QGW3"</field>Coop. Corp. Law &s;§62.\nResearch References\n ►<field>rc.ref.id="I05NNR7"</field>N.Y. Jur. 2d, Business Relationships &s;§789\nARTICLE III. \nSection 8. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>Any member may bring charges against a director by filing them in writing with the secretary, together with a petition signed by 5&percnt;% of the members, requesting removal. The corporation may thereupon remove the director by the affirmative vote of three-fourths of the members voting thereon at a meeting promptly held after due notice in writing, setting forth accurately the purpose for which such meeting is called, provided that at such meeting not less than 10&percnt;% of the entire membership vote, personally or by mail. The director involved shall be given a copy of the charges reasonably in advance of the meeting, and he or she and the complainant shall have an opportunity at the meeting to be heard in person or by counsel and to present and cross-examine witnesses. ►<field>st.ref.id="I05KUMB"</field>Coop. Corp. Law &s;§63.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>In case the bylaws provide for election of directors by districts with primary elections in each district, then the petition for removal of a director must be signed by 20&percnt;% of the members residing in the district from which such director was elected. The board of directors must call a special meeting of the members residing in that district to consider the removal of the director, and, by a concurrent vote of a majority of the members of the district voting on the issue, the director will be removed from office. ►<field>st.ref.id="I05QGW0"</field>Coop. Corp. Law &s;§63.\nResearch References\n ►<field>rc.ref.id="I05NNR4"</field>N.Y. Jur. 2d, Business Relationships &s;§569\nARTICLE IV. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>The officers of every cooperative corporation must include a president, one or more vice presidents, secretary, and treasurer who are required to be elected annually by the board of directors. The president and a first vice president must be members of the corporation or of a member corporation and must be elected from among the directors. Other officers need not be directors or members of the corporation. The offices of secretary and treasurer may be combined. ►<field>st.ref.id="I05KUM8"</field>Coop. Corp. Law &s;§64.\nResearch References\n ►<field>rc.ref.id="I05I1HC"</field>N.Y. Jur. 2d, Business Relationships &ss;§§532, 536\nARTICLE IX. \n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|24|32:24"</field>For an agricultural cooperative corporation, the bylaws may provide for their amendment by the board of directors; however, any amendment adopted by the board that affects the preferential rights or obligations of the members or stockholders shall be reported to the next annual meeting of the corporation and, if not affirmatively approved thereat, shall cease to be in effect. Such bylaws reported to the annual meeting may be adopted, repealed, or amended on the affirmative vote of two-thirds of the members, stockholders, or delegates voting thereon at a meeting held after due written notice setting forth the proposed action and the purpose of the meeting. ►<field>st.ref.id="I05F8CG"</field>Coop. Corp. Law &s;§111.\nResearch References\n ►<field>rc.ref.id="I05QGVY"</field>N.Y. Jur. 2d, Business Relationships &s;§128\n\n32:24.50. Bylaw provision-- Membership \nA membership in <finstr>[name of corporation]</finstr> is defined as any person, firm, corporation or other organization whose members are engaged in <finstr>[specification of activity]</finstr> provided the individual or corporation is contributing to or otherwise financially supporting the activities of this organization in a manner established and approved by the Board of Directors.\nEach member is entitled to one vote at the <finstr>[name of corporation]</finstr> membership meetings.\n\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I05KUGI"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05I1BM"</field>West\'s Key Number Digest, Associations &key;141\n\n\n32:25. Bylaw provision-- Amendment of bylaws\nCLS\nCOL\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3977fd0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|25|32:25"</field>These bylaws may be amended at any special or annual members\' meeting by two-thirds of the votes cast, provided that notice of the proposed alteration or amendment be contained in the notice of the meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|25|32:25"</field>These bylaws may be amended by the board of directors, provided, however, that any amendment adopted by the board that affects the preferential rights or obligations of the members or stockholders shall be reported to the next annual meeting of the corporation and, if not affirmatively approved thereat, shall cease to be in effect. Such bylaws reported to the annual meeting may be adopted, repealed, or amended on the affirmative vote of two-thirds of the members, stockholders, or delegates voting thereon at a meeting held after due written notice, setting forth the proposed action and the purpose of the meeting.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|25|32:25"</field>Bylaws may be adopted, repealed, or amended on the affirmative vote of two-thirds of the members, stockholders, or delegates voting thereon at a meeting held after due written notice setting forth the proposed action and the purpose of the meeting. ►<field>st.ref.id="I05QGQA"</field>Coop. Corp. Law &s;§16.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|25|32:25"</field>For an agricultural cooperative corporation, the bylaws may provide for their amendment by the board of directors, but any amendment adopted by the board that affects the preferential rights or obligations of the members or stockholders shall be reported to the next annual meeting of the corporation and, if not affirmatively approved thereat, shall cease to be in effect. Such bylaws reported to the annual meeting may be adopted, repealed, or amended on the affirmative vote of two-thirds of the members, stockholders, or delegates voting thereon at a meeting held after due written notice setting forth the proposed action and the purpose of the meeting. ►<field>st.ref.id="I05NNLE"</field>Coop. Corp. Law &s;§111.\nResearch References\n ►<field>tk.ref.id="I05KUGI"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05I1BM"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I05F86Q"</field>N.Y. Jur. 2d, Business Relationships &s;§128\n\n32:25.50. Bylaw provision—Principal office\nThe Board of Directors shall establish the principal office of the corporation as the headquarters for the conduct of activities. The Board of Directors may change the principal office when necessary and desirable.\n\nResearch References\n ►<field>tk.ref.id="I04BY0Y"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04ER5U"</field>West\'s Key Number Digest, Associations &key;128, 129\n\n32:26. Bylaw provision-- Waiver of requirements as to call, notice, and the like for board meetings\nCLS\nCOL\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia397cdf0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>Directors may waive any and all provisions of law and/or of these bylaws in respect of call, notice, and place of meeting, or any of them, and may consent to the holding of any meeting without call and notice, or either of them, and without regard to the place where held. Any director may waive call, notice, and place of meeting, or any of them, in respect of himself or herself and may consent to the holding of any meeting without call and notice, or either of them, and without regard to the place where held.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>Any regular or special meeting of directors, held with unanimous consent or approval of all the shareholders of the board and attended by not less than a quorum, shall be valid without regard to call and notice, or either of them, and without regard to the place where held. The proceedings of any such meeting shall be valid and constitute the act or acts of the board of directors as fully and conclusively as though taken at a meeting duly called and noticed and held at the place duly appointed. Attendance at and participation in the proceedings of any meeting, attended and participated in by not less than a quorum, without objection to the regularity or sufficiency of, or lack of, call and notice, or either, or the place where held, shall constitute and be a waiver in respect of such matters and an irrevocable consent to the holding of such meeting on the part of and by the director so attending and participating without objection.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>The waiver and consent of any director who is not present at such meeting may be made or given either before, during, or after the meeting and may be made and evidenced:\n \n►<field>fi.lvl="1"</field>(1). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>In writing, filed with the secretary, either before, during, or after the meeting;\n►<field>fi.lvl="1"</field>(2). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>By subscribing at the foot of the minutes of the meeting an approval of such minutes; or\n►<field>fi.lvl="1"</field>(3). \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>By approving, without objection, the minutes of the meeting at a subsequent meeting of the board, attended by the director who was absent from the first-mentioned meeting and whose waiver of consent is to be secured.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>One of the meetings of the corporation must be designated as the annual meeting. ►<field>st.ref.id="I05QGQ8"</field>BCL &s;§602(c).\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|26|32:26"</field>Waiver of notice of director\'s meetings is authorized by ►<field>st.ref.id="I09SLZQ"</field>BCL &s;§711(c).\nResearch References\n ►<field>tk.ref.id="I05I1BI"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I0494NP"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I04BXSL"</field>N.Y. Jur. 2d, Business Relationships &s;§600\n32:27. Bylaw provision-- Voting rights, distributions, and disposition of membership\nCLS\nCOL\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia397f500181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|27|32:27"</field>Each member shall be entitled to one vote regardless of the number of shares owned. The cooperative\'s net income derived from business with or for the members shall be allocated and distributed as patronage dividends to them in proportion to the relative amount of business done with or for each member. Patronage dividends will be distributed either in cash or in the form of qualified checks, as defined in ►<field>st.ref.id="I04EQXH"</field>Internal Revenue Code &s;§1388(b), no later than the [ordinal number of day] day of the [quarter/fiscal quarter/year] following the year in which earnings are derived. Dividends attributable to any nonpatronage income from business done with nonmembers, when distributed, will be distributed to the members in proportion to historical patronage.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|27|32:27"</field>Members shall not be permitted to sell their interest in the cooperative except to new members in connection with the admission of such new members or to the cooperative in connection with the withdrawal or expulsion of a member or for purposes of balancing ownership interests of the cooperative. If additional members are admitted to join the cooperative, pursuant to approval by a majority of existing members, the new member will be required to contribute cash as working capital to the cooperative in an amount determined by the members to be proportionate with the relative value of stock received and in relation to anticipated patronage by the new member.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|27|32:27"</field>For a discussion of considerations to be taken into account when drafting bylaws of a cooperative, see ►<field>x.ref.id="I0494OU"</field>&s;§32:11.\nResearch References\n ►<field>tk.ref.id="I04BXTQ"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04EQYM"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I04HK3I"</field>Construction and effect of corporate bylaws or articles relating to change in number of directors, 3 A.L.R.3d 623\n32:28. Bylaw provision-- Limiting shares owned, held, or voted\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3981c10181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|28|32:28"</field>The maximum number of shares permitted to be [owned/held/voted] by any one shareholder at any one time is [number of shares].\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I0494OX"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04BXTT"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I04EQYP"</field>N.Y. Jur. 2d, Business Relationships &s;§119\n32:29. Bylaw provision-- Compensation of directors\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3984320181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|29|32:29"</field>Directors shall receive no stated salary for their services as directors, but each director shall be paid for each regular or special meeting the sum of $[dollar amount of compensation] or such further sum as the members may fix from time to time at any meeting of the board. A director shall be allowed reasonable expenses while engaged in the business of the cooperative, to be audited, allowed, and paid as other claims against the cooperative. No director shall be appointed or employed by the cooperative for any position or in any capacity for which he or she shall be paid a salary or other remuneration for more than [number of days] days\' work in any one year, exclusive of the compensation and expenses to be paid for attendance at directors\' meetings.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|29|32:29"</field>A corporation may provide a fair remuneration for its officers and directors and for members of its executive committee. No officer or director or member of the executive committee may be a party to a contract for profit with the corporation that in any way differs from similar contracts with members or with others or that varies from terms generally current in the district. ►<field>st.ref.id="I04HK3L"</field>Coop. Corp. Law &s;§62.\nResearch References\n ►<field>rc.ref.id="I0494UK"</field>N.Y. Jur. 2d, Business Relationships &s;§789\n32:30. Bylaw provision-- Consent of members to take into account as income amounts distributed by notices of allocation\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3986a30181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|30|32:30"</field>Each person who hereafter applies for and is accepted to membership in this corporation, and each member of this corporation on the effective date of this article of the bylaws of the corporation who continues as a member after such date shall, by such act alone, consent that the amount of any distribution of patronage dividends [OPTIONAL: as defined in ►<field>st.ref.id="I0494UN"</field>Section 1388(a) of the Internal Revenue Code] and the amount of any distribution of the kind referred to in ►<field>st.ref.id="I04BXZJ"</field>Section 1382(c)(2)(A) of the Internal Revenue Code occurring after [identification of date], which distributions are made in written notices of allocation as defined in ►<field>st.ref.id="I04ER4F"</field>Section 1388(b) of the Internal Revenue Code and which are received by such person from the corporation, will be taken into account by such person at their stated dollar amount in the manner provided in ►<field>st.ref.id="I04HK9B"</field>Section 1385(a) of the Internal Revenue Code in the taxable year in which such written notices of allocation are received by such person.\nNotes to Form\n\nTax Notes \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|30|32:30"</field>This bylaw provision is included to qualify the written notices of allocation of the corporation and is substantially in the form found in the regulations. See ►<field>st.ref.id="I04KDE7"</field>26 C.F.R. &s;§1.1388-1(c)(3)(ii)(b). For the bylaw to be effective, the patron must receive written notification and a copy of the bylaw provision.\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|30|32:30"</field>For the bylaw to be effective, the patron must receive a written notification and copy of the bylaw. ►<field>st.ref.id="I0494UP"</field>26 U.S.C.A. &s;§1388(c)(2)(B)(ii).\nResearch References\n ►<field>tk.ref.id="I04BXZL"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04ER4H"</field>West\'s Key Number Digest, Associations &key;128, 129\n Noncash patronage dividends, ►<field>rc.ref.id="I04HK9D"</field>Federal Tax Guide to Legal Forms &s;§5:18\n\n32:31. Bylaw provision-- Election of directors by district\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3989140181a11e68738d3db3a7c8dd3"</field>\n►<field>fi.lvl="1"</field>Section 1. Election Districts. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|32|32:31"</field>The territory in which the corporation has members shall be divided into districts. Unless and until changed as herein provided, there shall be [number of districts] districts, and the respective districts and their boundaries shall be those established by resolution of the board of directors.\n►<field>fi.lvl="1"</field>Section 2. Change of Election Districts. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|32|32:31"</field>By resolution adopted by majority vote, the board of directors may, from time to time: (a) without affecting the number of districts, change the boundaries of one or more districts by adding to it territory not included up to that time within any district or by adding to one district territory theretofore embraced within another or by excluding from a district a part of its territories; or (b) change the number of districts by redistricting the entire territory, discontinuing a district, or merging a district with one or more districts or by any other means.\n►<field>fi.lvl="1"</field>Section 3. Election of Directors by District and At-Large. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|32|32:31"</field>There shall be as many directors as there are districts, one director to be elected by the members of each district, provided, however, that when the number of districts is an even number, there shall be one additional director to be known as a director-at-large and to be elected by all members of the corporation. Each director must be a member of the corporation and engaged in [description of activity] on a commercial scale or must be the authorized representative of a member so engaged if the latter is not a single natural person. A district director must be a resident of the district in which he or she is elected or for which he or she is appointed.\n►<field>fi.lvl="1"</field>Section 4. Effect on Tenure of Directors by Change in Districts. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|32|32:31"</field>Whenever a district is discontinued, the director representing that district shall cease to be a director and the office shall be discontinued. Whenever a new district is created, a director to represent it shall be selected in the manner herein provided in case of a vacancy. The same procedure is applicable when the creation of a new district necessitates the selection of a director-at-large. Notwithstanding the simultaneous discontinuance and creation of districts, if a director-at-large is still necessary as before such changes, the person then serving as such director shall continue to hold that office, subject to any other relevant provisions of the bylaws. Any question as to the effect of any changes made as herein provided in district boundaries or the number or identity of districts shall be conclusively determined by the board of directors.\n►<field>fi.lvl="1"</field>Section 5. Nomination of Directors. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|32|32:31"</field>Nominations for directors, either at-large or for a district, shall be made by petition addressed to the secretary of the corporation requesting that the secretary place upon the ballot the name of the person so nominated. Such a petition nominating a director-at-large shall be signed by not less than [fraction of members] members of the corporation. Such a petition nominating a district director shall be signed by not less than [fraction of members] of members from that district.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|32|32:31"</field>Election of directors according to territorial districts is provided for in ►<field>st.ref.id="I0494VX"</field>Coop. Corp. Law &s;§61.\nResearch References\n ►<field>tk.ref.id="I04BY0T"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04ER5P"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I0494VZ"</field>N.Y. Jur. 2d, Business Relationships &s;§545\n32:32. Bylaw provision-- Treatment of losses\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia398df60181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|33|32:32"</field>In the event of a substantial loss to the corporation from any cause whatsoever, the board of directors may, in its discretion, charge all or a part of such loss to current operating expenses. Any part of such loss not charged to current operating expenses (up to the whole amount of such loss) may, in the board\'s discretion, be: (1) charged ratably and proportionately against revolving fund credits for the year during which the loss occurred; or (2) charged ratably and proportionately against all revolving fund credits (for all years), in which event such loss may be apportioned between revolving funds.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|33|32:32"</field>In the event any loss is charged against revolving fund credits, such credit shall be reduced by its proportionate part of the loss and the records pertaining to the revolving fund changed accordingly. Notwithstanding anything to the contrary contained elsewhere in these bylaws, there shall be payable in respect to any credit against which a loss has been charged only the difference between the amount of the credit as originally entered and the portion of the loss charged to such credit.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|33|32:32"</field>The board of directors shall have power to determine when a substantial loss has occurred, and its determination shall be conclusive. In making such determination, the board may consider losses from bank suspension, uncollected accounts, fire, explosion, accident, or other calamity and losses arising from any other source or for any other reason.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|33|32:32"</field>A revolving fund operates to retain the capital of a cooperative; members of the cooperative are credited with their proportionate part of the fund. See ►<field>rc.ref.id="I0494W2"</field>Am. Jur. 2d, Cooperative Associations &s;§22.\nResearch References\n ►<field>tk.ref.id="I04BY0Y"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04ER5U"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I04HKAQ"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\n ►<field>rc.ref.id="I04KDFM"</field>N.Y. Jur. 2d, Business Relationships &s;§245\n32:32.50. Bylaw provision-- Liability \nThe amount of corporate indebtedness for which the individual members or directors of the corporation will personally be liable will not exceed [dollar amount of sum], all as provided by Section 47 of the Cooperative Corporations Law and the Certificate of Incorporation of this corporation. Any and all creditors must look only to the Corporation\'s assets for payment. \n\nResearch References\n ►<field>tk.ref.id="I04BY0Y"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04ER5U"</field>West\'s Key Number Digest, Associations &key;128, 129\n\n32:33. Bylaw provision-- Death of stockholder\nCLS\nCOL\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3990670181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|34|32:33"</field>The Board of Directors may repurchase at par value, plus any declared and unpaid dividends, the common capital stock of any deceased member. Should the Cooperative not desire to repurchase such stock of a deceased member it may be sold or transferred to any other person eligible for membership in the Corporation as set forth in these bylaws.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|34|32:33"</field>The board of directors shall not buy the member\'s stock for more than par value.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|34|32:33"</field>In accordance with provisions in the bylaws, any person shall forfeit his or her membership upon proof that he or she has ceased to be engaged in the occupation or occupations for the servicing of which the corporation was formed or has ceased to have the qualifications requisite for membership and shall, upon such notice and terms as may be prescribed in the bylaws, surrender his or her membership certificate or his or her shares of stock upon payment of the par or otherwise designated value thereof and of any accrued dividends thereon, as may appear in the accounting at the end of the current fiscal year. ►<field>st.ref.id="I05QGC3"</field>Coop. Corp. Law &s;§42.\nResearch References\n ►<field>tk.ref.id="I05NN77"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05KU2B"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I05I0XF"</field>N.Y. Jur. 2d, Business Relationships &s;§836\n32:34. Bylaw provision-- Change of residence of stockholder\nCLS\nCOL\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3992d80181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|35|32:34"</field>If any stockholder of the cooperative moves out of the locality, the directors shall repay to stockholder the face value of the capital stock owned by the stockholder, with all accrued interest and patronage dividends, within 12 months after the absence of the stockholder.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|35|32:34"</field>In accordance with provisions in the bylaws, any person shall forfeit his or her membership upon proof that he or she has ceased to be engaged in the occupation or occupations for the servicing of which the corporation was formed or has ceased to have the qualifications requisite for membership and shall, upon such notice and terms as may be prescribed in the bylaws, surrender his or her membership certificate or his or her shares of stock upon payment of the par or otherwise designated value thereof and of any accrued dividends thereon, as may appear in the accounting at the end of the current fiscal year. ►<field>st.ref.id="I05QGC0"</field>Coop. Corp. Law &s;§42.\nResearch References\n ►<field>tk.ref.id="I05NN74"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05KU28"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I05QGBY"</field>N.Y. Jur. 2d, Business Relationships &s;§836\n32:35. Bylaw provision-- Expulsion and withdrawal\nCOL\nCLS\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3995490181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|36|32:35"</field>The board of directors may prefer charges against any member in accordance with the bylaws if the board determines that such member has ceased to be a bona fide producer of [name of product], has violated any of the provisions of the bylaws or of the contract with the corporation, or has been guilty of conduct detrimental to the corporation.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|36|32:35"</field>Any member may withdraw from the corporation by giving [number of days] days\' notice in writing to the directors, but such withdrawal shall not affect any right or lien that the corporation has against the retiring member or the member\'s property until the indebtedness to the corporation is fully paid. If such notice is not given, it shall be assumed that each member desires to continue membership.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|36|32:35"</field>In accordance with provisions in the bylaws, any person shall forfeit his or her membership upon proof that he or she has ceased to be engaged in the occupation or occupations for the servicing of which the corporation was formed or has ceased to have the qualifications requisite for membership and shall, upon such notice and terms as may be prescribed in the bylaws, surrender his or her membership certificate or his or her shares of stock upon payment of the par or otherwise designated value thereof and of any accrued dividends thereon, as may appear in the accounting at the end of the current fiscal year. ►<field>st.ref.id="I05NN72"</field>Coop. Corp. Law &s;§42.\nResearch References\n ►<field>tk.ref.id="I05KU26"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05I0XA"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I05F7SE"</field>N.Y. Jur. 2d, Business Relationships &s;§836\n32:36. Bylaw provision-- Expulsion of member; Procedure\nCLS\nCOL\nFCV\nFTR\nCOR\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia3997ba0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|37|32:36"</field>Members may be expelled from membership for breach of contract or for acts injurious to the association.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|37|32:36"</field>Any member of this association may be expelled by a resolution passed and adopted at any meeting of the members called for the purpose or at any regular meeting, the reason for expulsion being stated in the resolution.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|37|32:36"</field>Three members of the board of directors or any officer shall notify the accused member, in writing, of the charge at least 15 days before the date of a proposed meeting when the charges shall be considered by the board of directors. Upon the affirmative vote of two-thirds of the directors and upon the recommendation of three executive officers, if any, of the member\'s respective district organization, which decision shall be final, the person so accused shall be denied further benefit of membership privileges except to conclude unfinished transactions.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|37|32:36"</field>The expelled member shall have his or her property interest in the association equitably and fairly appraised by the board of directors and the amount fixed in money within [number of days] days after the close of the season in which expelled, and there shall be paid to the member the amount of the appraisal as and when the same would have been paid had the person remained a member of the association. Furthermore, the appraisal of the interest of the member as made by the board of directors shall be conclusive and binding.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|37|32:36"</field>In accordance with provisions in the bylaws, any person shall forfeit his or her membership upon proof that he or she has ceased to be engaged in the occupation or occupations for the servicing of which the corporation was formed or has ceased to have the qualifications requisite for membership and shall, upon such notice and terms as may be prescribed in the bylaws, surrender his or her membership certificate or his or her shares of stock upon payment of the par or otherwise designated value thereof and of any accrued dividends thereon, as may appear in the accounting at the end of the current fiscal year. ►<field>st.ref.id="I05QGBV"</field>Coop. Corp. Law &s;§42.\nResearch References\n ►<field>tk.ref.id="I05NN6Z"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05QGAQ"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I05NN5U"</field>N.Y. Jur. 2d, Business Relationships &s;§836\n32:37. Bylaw provision-- Distribution on dissolution\nCOL\nCLS\nFCV\nFTR\nCOR\nREO\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia399a2b0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|39|32:37"</field>On dissolution of this corporation, all debts and liabilities of the corporation shall be paid first according to their respective priorities. Any property remaining after discharge of the corporation\'s debts and liabilities shall be distributed to the shareholders and patrons. Holders of shares of preferred stock shall first be paid and shall be limited to the par value of their preferred shares, plus any dividends declared thereon and unpaid. Holders of shares of common stock shall next be paid and shall be limited to the par value of their common shares. Holders of credits in the revolving fund shall then be paid in the amount of their credit in the fund. The remainder of the property shall be distributed among the patrons who have patronized the corporation during the [number of fiscal years] fiscal years immediately preceding the dissolution, on the basis of their respective patronage as shown by the corporation\'s records.\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|39|32:37"</field>A cooperative corporation may, at any meeting and upon due and express notice previously given, by vote of two-thirds of all of the members or stockholders voting thereon, discontinue its operations and settle its affairs. See ►<field>st.ref.id="I05QGAO"</field>Coop. Corp. Law &s;§17.\nResearch References\n ►<field>tk.ref.id="I05NN5S"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05KU0W"</field>West\'s Key Number Digest, Associations &key;128, 129\n ►<field>rc.ref.id="I04A8Y4"</field>N.Y. Jur. 2d, Business Relationships &ss;§§1337, 1432\n3. Meetings\n32:38. Notice of first meeting of board of directors\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia399f0d0181a11e68738d3db3a7c8dd3"</field>►<field>fcap.ref="manual"</field>►<field>tbl.date.updated="0"</field><field>tbl.orient="port"</field><field>tbl.position="l"</field><field>tbl.table.width="100"</field><field>tbl.uuid="I039XNG"</field>\n►<field>table.resize="same"</field><field>table.tabstyle="text"</field>\n►<field>tgroup.cols="2"</field>\n►<field>colspec.colname="1"</field><field>colspec.colnum="1"</field><field>colspec.colwidth="1*"</field>\n►<field>colspec.colname="2"</field><field>colspec.colnum="2"</field><field>colspec.colwidth="19*"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>To: \n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>[Name of director]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>[Address of director]\n\n\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|40|32:38"</field>You are notified that the first meeting of the board of directors of [name of cooperative corporation] will be held on [date of meeting], at [time of meeting], at [address of meeting], [name of city], [name of county], State of New York, for the purposes of electing corporate officers for the year [identification of year], adopting the form of marketing agreement to be used between the corporation and its members, and [specification of other business], and transacting any other business that may properly come before the meeting.\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of notice]\n►<field>fsig.pos="r"</field>►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of person issuing notice]\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I04D230"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04A8Y6"</field>West\'s Key Number Digest, Associations &key;171\n32:39. Waiver of notice of first meeting of board of directors\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39a17e0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|41|32:39"</field>We, the undersigned, being all the directors of [name of cooperative corporation] of [address of cooperative corporation], [name of city], [name of county], State of New York, waive notice of a meeting of the directors and consent to the holding of a meeting of such directors at [time of meeting], on [date of meeting], at [address of meeting], [name of city], [name of county], State of New York, for the purpose of electing corporate officers to serve during the ensuing year, [specification of other business], and of transacting any other business that may properly come before the meeting.\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of waiver]\n►<field>fsig.pos="r"</field>►<field>fal.lo="y"</field> ______________ \n\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Names of directors]\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I04D232"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04A8Y9"</field>West\'s Key Number Digest, Associations &key;171\n32:40. Waiver of notice of first meeting of members\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39a3ef0181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|42|32:40"</field>We, the undersigned, being all of the incorporators of [name of corporation], of [name of city], New York, constituting all of the present members of the corporation, waive notice of a meeting of the members and consent to the holding of a meeting of such members at [time of meeting] on [date of meeting], at [place of meeting] in [name of city], New York, for the purpose of adopting bylaws for the government of the corporation and transacting any other business that may properly come before the meeting.\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of waiver]\n►<field>fsig.pos="r"</field>►<field>fal.lo="y"</field> ______________ \n\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Names of incorporators]\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I04D235"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04FV81"</field>West\'s Key Number Digest, Associations &key;171\n32:41. Minutes of first meeting of board of directors\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39a6600181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>The first meeting of the board of directors of [name of cooperative corporation], of [address of cooperative corporation], [name of city], [name of county], State of New York, was held at [time of meeting], on [date of meeting], at [address of meeting], [name of city], [name of county], State of New York.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>On convening, [name of temporary chairperson] was elected temporary chairperson and [name of temporary secretary] temporary secretary of the meeting, and each assumed [his/her] office.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>The chair called for proof of notice of the meeting, whereupon [name of person presenting waiver of notice] presented a waiver of notice and consent to hold the meeting, signed by all the directors of the corporation, which waiver and consent was in the following form: [recitation of waiver of notice and consent to holding of meeting].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>On roll call of the directors of the corporation, the following answered present: [names of all directors present].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>The chair ruled that proper and legal notice of the meeting had been given and that a quorum was present and announced that the meeting was opened for the transaction of business.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>The chair stated that the meeting was called for the purpose of electing corporate officers for the ensuing year and for the transaction of any other business that might properly come before the meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>On motion duly made and seconded, the following officers were unanimously elected to serve at the board\'s discretion until the time of the board\'s first regular meeting, to be held as soon as practicable following the first annual shareholder\'s meeting: [names of officers elected] and [titles of officers elected].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>Following the election of officers, the president took the chair, and the secretary assumed [his/her] duties as secretary on the meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>On motion duly made, seconded, and carried, the following resolution was adopted:\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>Resolved, that [recitation of text resolution].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>[Description of additional business transacted].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|43|32:41"</field>There being no further business to come before the meeting, on motion duly made, seconded, and unanimously adopted, the meeting was adjourned.\n ►<field>fsig.pos="r"</field>►<field>fal.lo="n"</field>[Name of corporation]\n ►<field>fal.lo="n"</field>By:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of authorized representative]\n ►<field>fal.lo="n"</field>[Title of authorized representative]\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Attested:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of secretary]\n ►<field>fal.lo="n"</field>Secretary\n ►<field>fal.lo="n"</field>[Corporate seal]\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I04A8ZH"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04D24D"</field>West\'s Key Number Digest, Associations &key;171\n32:42. Minutes of first meeting of members\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39a8d10181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|44|32:42"</field>The first meeting of the members of [name of corporation] was held at [time of meeting], on [date of meeting], at [place of meeting], in [name of city], New York. The chair called for proof of notice of the meeting, whereupon [name of person presenting waiver and consent] presented a waiver of notice and consent to hold the meeting signed by all of the members of the corporation, which waiver and consent was in the following form:\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|44|32:42"</field>[Text of waiver of notice and consent to meeting].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|44|32:42"</field>The chair ruled that the meeting was properly called and it was ascertained that all the members of the corporation were present. The chair reported that the articles of incorporation of the corporation were filed on [date of filing articles of incorporation], at [time of filing articles of incorporation], in the office of the Secretary of State of the State of New York, and presented a copy of the articles of incorporation, which was read, and on motion duly made, seconded and carried, was directed to be entered in full in the minute book. (See page [number of page]). A draft of proposed bylaws for the government of the corporation was presented by [name of presenter] and was read to the meeting and discussed section by section and as a whole, and the proposed bylaws were unanimously adopted. Each member affixed his or her signature to the bylaws and the secretary was instructed to spread the bylaws on the minutes of this meeting.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|44|32:42"</field>[Description of record of other business that may have been transacted].\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I04FV99"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04A8ZJ"</field>West\'s Key Number Digest, Associations &key;171\n4. Certificates\n32:43. Certificate of membership\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n►<field>form.du="0"</field><field>form.fid="Ia39adb30181a11e68738d3db3a7c8dd3"</field>MEMBERSHIP CERTIFICATE \n►<field>fvs.amt="1"</field>\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|45|32:43"</field>This certifies that [name of member] of [address of member], [name of city], [name of state] is a Member of [name of cooperative] and is entitled to all of the rights, benefits, and privileges of the Cooperative.\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of certificate]\n►<field>fsig.pos="r"</field>►<field>fal.lo="y"</field> ______________ \n[Name of officer]\n ►<field>fal.lo="n"</field>[Title of officer]\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|45|32:43"</field>Every cooperative corporation shall issue to each member, upon full payment therefor, a certificate of membership (whether evidenced by stock or otherwise) that shall not be transferable otherwise than as may be prescribed in the Cooperative Corporations Law, the certificate of incorporation, and bylaws. The corporation may accept as full or partial payment a member\'s promissory note but shall hold the certificate as security for payment, without, however, affecting the member\'s right to vote unless such note is past due. ►<field>st.ref.id="I04A8ZM"</field>Coop. Corp. Law &s;§40.\nResearch References\n ►<field>tk.ref.id="I04D24I"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04FV9E"</field>West\'s Key Number Digest, Associations &key;141 to 160\n ►<field>rc.ref.id="I04IOEA"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\n ►<field>rc.ref.id="I04LHJ6"</field>N.Y. Jur. 2d, Business Relationships &s;§825\n32:44. Certificate of membership-- Payment of membership fee\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39b5060181a11e68738d3db3a7c8dd3"</field>►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|46|32:44"</field>This is to certify that [name of member] of [address of member], [name of city], [name of county], [name of state], is a member of [name of cooperative corporation] and is entitled to the rights and privileges of membership and is likewise bound by and subject to the obligations and conditions pertaining thereto as set forth in the certificate of incorporation, bylaws, and marketing agreement, now or hereafter in effect. The member has paid a membership fee of $[dollar amount of membership fee].\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|46|32:44"</field>This certificate and the membership and the rights represented hereby are nontransferable, except as provided in the bylaws.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|46|32:44"</field>In witness, [name of cooperative corporation] has caused the certificate to be signed by its duly authorized officers and its corporate seal to be affixed on [date of certificate].\n ►<field>fsig.pos="r"</field>►<field>fal.lo="n"</field>[Name of corporation]\n ►<field>fal.lo="n"</field>By:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of authorized representative]\n ►<field>fal.lo="n"</field>[Title of authorized representative]\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Attested:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of secretary]\n ►<field>fal.lo="n"</field>Secretary\n ►<field>fal.lo="n"</field>[Corporate seal]\nNotes to Form\n\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|46|32:44"</field>Every cooperative corporation shall issue to each member, upon full payment therefor, a certificate of membership (whether evidenced by stock or otherwise) that shall not be transferable otherwise than as may be prescribed in the Cooperative Corporations Law, the certificate of incorporation, and bylaws. The corporation may accept as full or partial payment a member\'s promissory note but shall hold the certificate as security for payment, without, however, affecting the member\'s right to vote unless such note is past due. ►<field>st.ref.id="I04A95B"</field>Coop. Corp. Law &s;§40.\nResearch References\n ►<field>tk.ref.id="I04D2A7"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04FVF3"</field>West\'s Key Number Digest, Associations &key;141 to 160\n ►<field>rc.ref.id="I04IOJZ"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\n ►<field>rc.ref.id="I04LHOV"</field>N.Y. Jur. 2d, Business Relationships &s;§825\n32:45. Patronage dividend certificate\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39b7770181a11e68738d3db3a7c8dd3"</field>►<field>fcap.ref="manual"</field>►<field>tbl.date.updated="0"</field><field>tbl.orient="port"</field><field>tbl.position="l"</field><field>tbl.table.width="100"</field><field>tbl.uuid="I03A6WE"</field>\n►<field>table.resize="same"</field><field>table.tabstyle="text"</field>\n►<field>tgroup.cols="2"</field>\n►<field>colspec.colname="1"</field><field>colspec.colnum="1"</field><field>colspec.colwidth="1*"</field>\n►<field>colspec.colname="2"</field><field>colspec.colnum="2"</field><field>colspec.colwidth="19*"</field>\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>To:\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>[Name of member]\n\n\n►<field>row.date.updated="0"</field>►<field>entry.resize="same"</field><field>entry.style="block"</field>\n\n►<field>entry.resize="same"</field><field>entry.style="block"</field>[Address of member]\n\n\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|47|32:45"</field>The corporation has allocated to your patronage dividend account the amount of $[dollar amount of patronage dividend], representing your share in the net profits of the corporation based on the amount of your purchases for the period [identification of year 1] to [identification of year 2]. The amount of your patronage dividend will be paid to you on [due date of payment].\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Dated: [date of certificate]\n ►<field>fsig.pos="r"</field>►<field>fal.lo="n"</field>[Name of corporation]\n ►<field>fal.lo="n"</field>By:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of authorized representative]\n ►<field>fal.lo="n"</field>[Title of authorized representative]\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Attested:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of secretary]\n ►<field>fal.lo="n"</field>Secretary\n ►<field>fal.lo="n"</field>[Corporate seal]\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I04A95E"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I04D2AA"</field>West\'s Key Number Digest, Associations &key;141\n ►<field>tk.ref.id="I04FVF6"</field>West\'s Key Number Digest, Associations &key;147\n ►<field>tk.ref.id="I04IOK2"</field>West\'s Key Number Digest, Associations &key;179\n ►<field>rc.ref.id="I04LHOY"</field>Co-operative associations: rights in equity credits or patronage dividends, 50 A.L.R.3d 435\n ►<field>rc.ref.id="I05PC8C"</field>Am. Jur. 2d, Cooperative Associations &s;§21\n32:46. Revolving fund certificate\nFMS\nCOL\nFCV\nFTR\nCOR\nEBL\nTEM\nMTD\n ►<field>form.du="0"</field><field>form.fid="Ia39bc590181a11e68738d3db3a7c8dd3"</field>►<field>fcap.ref="manual"</field>►<field>fal.lo="n"</field>Series Date [Date of series]\n ►<field>fal.lo="n"</field>No. [Number of certificate]\n ►<field>fal.lo="n"</field>Amount: $[Dollar amount of sum payable]\n \n►<field>fi.lvl="1"</field>[Name of Corporation]\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|48|32:46"</field>This certifies that [name of member] of [address of member], [name of city], [name of county], State of New York, is entitled to receive the amount of $[dollar amount of sum payable] from [name of corporation] on account of capital furnished and patronage dividends or deductions for revolving fund purposes as provided in the corporation\'s bylaws, subject to the following conditions:\n \n►<field>fi.lvl="2"</field>1. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|48|32:46"</field>This and other revolving fund certificates of the same series are retireable at the sole discretion of the board of directors, either fully or on a pro rata basis, but certificates issued in prior years shall be entitled to priority in retirement, except in liquidation.\n►<field>fi.lvl="2"</field>2. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|48|32:46"</field>The amount stated in the certificate shall bear only the rate of interest, if any, as the board of directors may fix, not to exceed [percentage rate of annual interest]&percnt;% per year.\n►<field>fi.lvl="2"</field>3. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|48|32:46"</field>This certificate is transferrable only on the books of the corporation.\n►<field>fi.lvl="2"</field>4. \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|48|32:46"</field>This and other certificates shall be subordinate to all other debts of the corporation, both secured and unsecured. On the winding up or liquidation of the corporation in any manner, after full payment to all its other creditors, all revolving fund certificates shall then be retired in full or on a pro rata basis without priority.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|48|32:46"</field>In witness, [name of corporation] has caused this certificate to be signed by its authorized officers and to be sealed with its seal at [place of execution] on [date of certificate].\n ►<field>fsig.pos="r"</field>►<field>fal.lo="n"</field>[Name of corporation]\n ►<field>fal.lo="n"</field>By:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of authorized representative]\n ►<field>fal.lo="n"</field>[Title of authorized representative]\n►<field>fvs.amt="1"</field>\n ►<field>fal.lo="n"</field>Attested:\n►<field>fvs.amt="1"</field>\n►<field>fal.lo="y"</field> ______________ \n[Name of secretary]\n ►<field>fal.lo="n"</field>Secretary\n ►<field>fal.lo="n"</field>[Corporate seal]\nNotes to Form\n\nResearch References\n ►<field>tk.ref.id="I05PC76"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05MJ2A"</field>West\'s Key Number Digest, Associations &key;141\n ►<field>tk.ref.id="I05JPXE"</field>West\'s Key Number Digest, Associations &key;147\n ►<field>tk.ref.id="I09QCXM"</field>West\'s Key Number Digest, Associations &key;179\n ►<field>rc.ref.id="I05MJ28"</field>Am. Jur. 2d, Cooperative Associations &s;§22\n5. Miscellaneous\n32:47. Application for recognition of exemption (IRS Form 1028)-- Official form\nCOL\nFCV\nFTR\nEFM\nCOR\nTXA\nTFD\nTF2\nEBL\nTEM\nMTD\n►<field>form.du="0"</field><field>form.fid="Ia39c13b0181a11e68738d3db3a7c8dd3"</field>►<field>ib.ct.id="6d0bf250b92611ef8747b569df12a67d|49|32:47"</field><field>ib.du="0"</field>►<field>im.format="pdf"</field><field>im.id="I000746"</field>\n►<field>im.format="tif"</field><field>im.id="I00016I"</field>\n ►<field>image.link.href="./Images/NYLB 32 (revision copy)-Images.pdf"</field>Image #I00016I goes here\n►<field>im.format="tif"</field><field>im.id="I0219H2"</field>\n ►<field>image.link.href="./Images/NYLB 32 (revision copy)-Images.pdf"</field>Image #I0219H2 goes here\n►<field>im.format="tif"</field><field>im.id="I01ZKTU"</field>\n ►<field>image.link.href="./Images/NYLB 32 (revision copy)-Images.pdf"</field>Image #I01ZKTU goes here\n►<field>im.format="tif"</field><field>im.id="I01WSHF"</field>\n ►<field>image.link.href="./Images/NYLB 32 (revision copy)-Images.pdf"</field>Image #I01WSHF goes here\nNotes to Form\n\nTax Notes \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|49|32:47"</field>An organization claiming exemption must file ►<field>st.ref.id="I05PC1G"</field>IRS Form 1028 with the district director for the internal revenue district in which the organization\'s principal place of business or principal office is located.\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|49|32:47"</field>►<field>st.ref.id="I05MIWK"</field>26 C.F.R. &s;§1.521-1(e).\n \n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|49|32:47"</field>The above form, revised September 2006, is available from the Internal Revenue Service, ►<field>url.ref.id="I05JPRO"</field>https://www.irs.gov, and is reprinted for illustrative purposes.\nResearch References\n ►<field>tk.ref.id="I05GWMS"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>tk.ref.id="I05PC1E"</field>West\'s Key Number Digest, Associations &key;101 et seq.\n Applying for exemption for farmers\' cooperatives, ►<field>rc.ref.id="I05MIWI"</field>Federal Tax Coordinator 2d &p;E-1043\n Exempt farmers\' cooperatives, ►<field>rc.ref.id="I05JPRM"</field>Federal Tax Guide to Legal Forms &s;§5:20']

In [34]:
print(part)

['II. Agricultural Cooperative Corporations\nA. Form Drafting Principles\n32:8. Introduction\nResearch References\n ►<field>tk.ref.id="I04AAC5"</field>West\'s Key Number Digest, Agriculture &key;6\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>A nonprofit cooperative agricultural corporation may be organized to engage in any cooperative activities for its members, including the marketing, selling of, bargaining, and contracting for agricultural products and the by-products of such products.1 Any five or more producers of agricultural products, to include cooperative corporations of such producers organized under the laws of New York or any other state, may form a nonprofit cooperative agricultural corporation, either with or without capital stock.2\n ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>If the principal activities of the corporation are connected with the marketing, processing, manufacture, or sale of agricultural products, it may be term

In [32]:
import string
designators = [f"{char}." for char in string.ascii_uppercase]
lines = parts[1].splitlines()
print(parts[1])

II. Agricultural Cooperative Corporations
A. Form Drafting Principles
32:8. Introduction
Research References
 ►<field>tk.ref.id="I04AAC5"</field>West's Key Number Digest, Agriculture &key;6
 ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>A nonprofit cooperative agricultural corporation may be organized to engage in any cooperative activities for its members, including the marketing, selling of, bargaining, and contracting for agricultural products and the by-products of such products.1 Any five or more producers of agricultural products, to include cooperative corporations of such producers organized under the laws of New York or any other state, may form a nonprofit cooperative agricultural corporation, either with or without capital stock.2
 ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>If the principal activities of the corporation are connected with the marketing, processing, manufacture, or sale of agricultural products, it may be termed a coop

In [31]:
   # Iterate over each line
for i,line in enumerate(lines):
    # Split the line by spaces and check the first word
    # if line.strip() == "":
    #     continue
    print(line)
    # first_word = line.strip().split(" ")[0]
    # print(first_word)
    # if first_word in designators:
    #     print("yes")

II. Agricultural Cooperative Corporations
A. Form Drafting Principles
32:8. Introduction
Research References
 ►<field>tk.ref.id="I04AAC5"</field>West's Key Number Digest, Agriculture &key;6
 ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>A nonprofit cooperative agricultural corporation may be organized to engage in any cooperative activities for its members, including the marketing, selling of, bargaining, and contracting for agricultural products and the by-products of such products.1 Any five or more producers of agricultural products, to include cooperative corporations of such producers organized under the laws of New York or any other state, may form a nonprofit cooperative agricultural corporation, either with or without capital stock.2
 ►<field>p.ct.id="6d0bf250b92611ef8747b569df12a67d|8|32:8"</field>If the principal activities of the corporation are connected with the marketing, processing, manufacture, or sale of agricultural products, it may be termed a coop